<a href="https://colab.research.google.com/github/daiki-nakajima-createor/Kaggle_CIBMTR/blob/main/cibmtr-eda-ensemble-model-efs-xgb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

このノートブックは、[Albanito](https://www.kaggle.com/albansteff)さんのノートブック[CIBMTR | EDA & Ensemble Model - Recalculate HLA](https://www.kaggle.com/code/albansteff/cibmtr-eda-ensemble-model-recalculate-hla)を元にしています。素晴らしい分析や洞察を共有していただきありがとうございます。

<p style="background-color: rgb(247, 230, 202); font-size: 300%; text-align: center; border-radius: 40px 40px; color: rgb(162, 87, 79); font-weight: bold; font-family: 'Roboto'; border: 4px solid rgb(162, 87, 79);">Introduction</p>

Addition in this notebook is the recalculation of HLA sums which are often missing. Since the sum is full of missing values, it was interesting to modify the values by recalculating them based on the data dictionary explanations and see the results.

このノートブックにおける追加事項は、しばしば欠けているHLA合計の再計算です。この合計は欠損値が多いため、データ辞書の説明に基づいて値を再計算し、その結果を確認することが興味深いものでした

In [1]:
# HLA : Human Leukocyte Antigen matching levels.
# Homozygous chromosomes have the same allele at a given locus (fixed position on a chromosome where a particular gene is located).
# Those that have different alleles at a given locus are called heterozygous.
# Values Explanation
# 0 - No Match: Neither of the donor's two HLA antigens/alleles matches the recipient's HLA.
# This indicates a complete mismatch at the locus, which increases the risk of complications like graft rejection or graft-versus-host disease (GVHD).
# 1 - Partial Match: One of the donor's HLA antigens/alleles matches one of the recipient's.
# This represents a half-match (heterozygous compatibility) at the locus. It's better than a full mismatch but still carries a moderate risk of immune complications.
# 2 - Full Match: Both of the donor's HLA match both of the recipient's HLA.
# This is the optimal scenario, indicating full compatibility at the locus and minimizing the risk of immune complications.

# High VS Low resolution

# High-Resolution Typing: Identifies specific alleles (e.g., HLA-A*02:01).
# Provides the most precise match and is essential for unrelated donor transplants.

# Low-Resolution Typing: Identifies broader antigen groups (e.g., HLA-A2).
# May suffice for related donor transplants where genetic similarity is inherently higher.

# HLA: ヒト白血球抗原の一致レベルについて
# ホモ接合染色体は、特定の遺伝子が位置する固定されたクロモソーム上の地点に同じ対立遺伝子を持っています。
# 異なる対立遺伝子を持つものはヘテロ接合と呼ばれます。
# 値の説明
# 0 - 不一致: ドナーの2つのHLA抗原/対立遺伝子が受容者のHLAと一致しない。
# これは、ラーカス（座位）での完全な不一致を示し、移植片拒絶や移植片対宿主病（GVHD）のような合併症のリスクを高めます。
# 1 - 部分一致: ドナーのHLA抗原/対立遺伝子の1つが受容者のものと一致する。
# これはラーカスでの半分の一致（ヘテロ接合の適合性）を表し、完全な不一致よりは良いですが、免疫の合併症のリスクは中程度あります。
# 2 - 完全一致: ドナーの両方のHLAが受容者の両方のHLAと一致する。
# これは最適なシナリオであり、ラーカスでの完全な適合性を示し、免疫合併症のリスクを最小限に抑えます。
# 高解像度 VS 低解像度
# 高解像度型: 特定の対立遺伝子を識別（例：HLA-A*02:01）。
# 最も正確な一致を提供し、非関連ドナーの移植には不可欠です。
# 低解像度型: より広い抗原グループを識別（例：HLA-A2）。
# 遺伝的類似性が本質的に高い関連ドナーの移植には十分である場合があります。

HLA_COLUMNS = [
    # MHC class I molecules are one of two primary classes of major histocompatibility complex (MHC) molecules and are found on the cell surface of all nucleated cells.
    # In humans, the HLAs corresponding to MHC class I are HLA-A, HLA-B, and HLA-C.

#     MHCクラスI分子は、主要組織適合性複合体（MHC）分子の2つの主要なクラスのうちの1つで、すべての有核細胞の細胞表面に存在します。
# ヒトでは、MHCクラスIに対応するHLAはHLA-A、HLA-B、HLA-Cです。
    'hla_match_a_low', 'hla_match_a_high',
    'hla_match_b_low', 'hla_match_b_high',
    'hla_match_c_low', 'hla_match_c_high',

    # MHC Class II molecules are a class of major histocompatibility complex (MHC) molecules normally found only on professional antigen-presenting cells
    # such as dendritic cells, macrophages, some endothelial cells, thymic epithelial cells, and B cells.
    # Antigens presented by MHC class II molecules are exogenous, originating from extracellular proteins rather than cytosolic and endogenous sources like
    # those presented by MHC class I.
    # HLAs corresponding to MHC class II are HLA-DP, HLA-DM, HLA-DOA, HLA-DOB, HLA-DQ, and HLA-DR.
    # In this competition, we only have HLA-DR and HLA-DQ

#     MHCクラスII分子は、通常、樹状細胞、マクロファージ、一部の内皮細胞、胸腺上皮細胞、B細胞などの専門の抗原提示細胞にのみ見られる主要組織適合性複合体（MHC）分子のクラスです。
# MHCクラスII分子によって提示される抗原は外来由来であり、細胞質や内因性のソース（MHCクラスIによって提示されるもの）ではなく、細胞外タンパク質から由来しています。
# MHCクラスIIに対応するHLAは、HLA-DP、HLA-DM、HLA-DOA、HLA-DOB、HLA-DQ、HLA-DRです。
# この競技会では、HLA-DRとHLA-DQのみがあります。

    # Visit https://en.wikipedia.org/wiki/HLA-DQB1

    'hla_match_dqb1_low', 'hla_match_dqb1_high',
    'hla_match_drb1_low', 'hla_match_drb1_high',

    # Combination of matches : sum of matches between multiple categories

    # Matching at HLA-A(low), -B(low), -DRB1(high)
    'hla_nmdp_6',
    # Matching at HLA-A,-B,-DRB1 (low or high)
    'hla_low_res_6', 'hla_high_res_6',
    # Matching at HLA-A, -B, -C, -DRB1 (low or high)
    'hla_low_res_8', 'hla_high_res_8',
    # Matching at HLA-A, -B, -C, -DRB1, -DQB1 (low or high)
    'hla_low_res_10', 'hla_high_res_10'
]

<div style="background-color: rgb(247, 230, 202); border: 4px solid rgb(162, 87, 79); border-radius: 40px; padding: 20px; font-family: 'Roboto'; color: rgb(162, 87, 79); text-align: left; font-size: 120%;">
    <ul style="list-style-type: square; padding-left: 20px;">
        <li>Missing values are replaced with:
            <ul style="list-style-type: circle; margin-top: 10px; margin-bottom: 10px;">
                <li>-1 for numeric columns</li>
                <li>Unknown for categorical columns</li>
            </ul>
        </li>
        <li style="margin-top: 10px;">
            LightGBM and CatBoost are trained on 3 different targets, estimated from the survival models:
            <ul style="list-style-type: circle; margin-top: 10px; margin-bottom: 10px;">
                <li>Cox</li>
                <li>Kaplan-Meier</li>
                <li>Nelson-Aalen</li>
            </ul>
        </li>
        <li style="margin-top: 10px;">Two additional CatBoost model are trained, with Cox loss function.</li>
        <li style="margin-top: 10px;">As per <a href="https://www.kaggle.com/competitions/equity-post-HCT-survival-predictions/discussion/553061" style="color: #A2574F; text-decoration: underline;">this</a> discussion post, the target is consisted of the Out-of-Fold predictions of the survival models on the validation folds to prevent target leakage.</li>
        <li style="margin-top: 10px;">
            The ensemble prediction for each sample is computed as:
            <ul style="list-style-type: circle; margin-top: 10px; margin-bottom: 10px;">
                <p style="margin-top: 10px; font-size: 110%; color: #A2574F; font-family: 'Roboto'; text-align: left;">
                    $ \text{preds}_{\text{ensemble}} = \sum_{i=1}^{n} w_i \cdot \text{rankdata}(\text{preds}_i) $
                </p>
                where $n$ is the number of models, $w_i$ is the weight assigned to the $i$-th model, and $\text{rankdata}(\text{preds}_i)$ is the rank of predictions from the $i$-th model.
            </ul>
        </li>
        <li style="margin-top: 10px;">Last but not least, since the competition metric evaluates only the order of predictions and not their magnitude, the model weights are not required to sum to 1, nor should the predictions fall within a predefined range.</li>
    </ul>
</div>

<div style="background-color: rgb(247, 230, 202); border: 4px solid rgb(162, 87, 79); border-radius: 40px; padding: 20px; font-family: 'Roboto'; color: rgb(162, 87, 79); text-align: left; font-size: 120%;">
    <ul style="list-style-type: square; padding-left: 20px;">
        <li>欠損値は次の値で置換されます：
            <ul style="list-style-type: circle; margin-top: 10px; margin-bottom: 10px;">
                <li>数値列には-1</li>
                <li>カテゴリ列には"Unknown"</li>
            </ul>
        </li>
        <li style="margin-top: 10px;">
            LightGBMおよびCatBoostは、生存モデルから推定された3つの異なるターゲットで訓練されます：
            <ul style="list-style-type: circle; margin-top: 10px; margin-bottom: 10px;">
                <li>Cox</li>
                <li>Kaplan-Meier</li>
                <li>Nelson-Aalen</li>
            </ul>
        </li>
        <li style="margin-top: 10px;">さらに2つのCatBoostモデルがCox損失関数で訓練されます。</li>
        <li style="margin-top: 10px;"><a href="https://www.kaggle.com/competitions/equity-post-HCT-survival-predictions/discussion/553061" style="color: #A2574F; text-decoration: underline;">この</a>ディスカッション投稿に従い、ターゲットは、バリデーションフォールドにおける生存モデルのOut-of-Fold予測で構成され、ターゲットリーケージを防止します。</li>
        <li style="margin-top: 10px;">
            各サンプルのアンサンブル予測は次のように計算されます：
            <ul style="list-style-type: circle; margin-top: 10px; margin-bottom: 10px;">
                <p style="margin-top: 10px; font-size: 110%; color: #A2574F; font-family: 'Roboto'; text-align: left;">
                    $ \text{preds}_{\text{ensemble}} = \sum_{i=1}^{n} w_i \cdot \text{rankdata}(\text{preds}_i) $
                </p>
                ここで $n$ はモデルの数、 $w_i$ は$i$番目のモデルに割り当てられた重み、$\text{rankdata}(\text{preds}_i)$ は$i$番目のモデルからの予測のランクです。
            </ul>
        </li>
        <li style="margin-top: 10px;">最後に、コンペティションの評価指標は予測の大きさではなく順位のみを評価するため、モデルの重みが1に合計される必要はなく、予測が事前に定義された範囲に収まる必要もありません。</li>
    </ul>
</div>

<p style="background-color: rgb(247, 230, 202); font-size: 300%; text-align: center; border-radius: 40px 40px; color: rgb(162, 87, 79); font-weight: bold; font-family: 'Roboto'; border: 4px solid rgb(162, 87, 79);">Install Libraries</p>

In [2]:
import os
if "KAGGLE_KERNEL_RUN_TYPE" in os.environ:
  !pip install /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
  !pip install /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
  !pip install /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
  !pip install /kaggle/input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
  !pip install /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl

Processing /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
autograd is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
  Preparing metadata (setup.py) ... done
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=54ba7774ce5442c5da9e0c2455caab34e8362d7e88385e895184f7754b3105e1
  Stored in directory: /root/.cache/pip/wheels/6b/b5/e0/4c79e15c0b5f2c15ecf613c720bb20daab20a666eb67135155
Successfully built autograd-gamma
Processing /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
Processing /kaggle/input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
Processing /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl


<p style="background-color: rgb(247, 230, 202); font-size: 300%; text-align: center; border-radius: 40px 40px; color: rgb(162, 87, 79); font-weight: bold; font-family: 'Roboto'; border: 4px solid rgb(162, 87, 79);">Imports</p>

In [3]:
import warnings
from pathlib import Path
warnings.filterwarnings('ignore')

In [4]:
import numpy as np
import polars as pl
import pandas as pd
import plotly.colors as pc
import plotly.express as px
import plotly.graph_objects as go

In [5]:
import plotly.io as pio
pio.renderers.default = 'iframe'

In [6]:
if "COLAB_GPU" in os.environ:
  !pip install lifelines
  !pip install catboost
  !pip install lightgbm

In [7]:
import lightgbm as lgb

from scipy.stats import rankdata
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
import joblib
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime
from scipy.stats import yeojohnson
import xgboost as xgb

# 全てのカラムを表示するオプションを設定
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [8]:
if "COLAB_GPU" in os.environ:
  import sys
  sys.path.append('/content/drive/MyDrive/Kaggle/20250215_CIBMTR')


  print("Google Colab で実行中")
  from google.colab import drive
  drive.mount('/content/drive')
  # CSVファイルのパスを指定
  csv_file_path = "/content/drive/MyDrive/Kaggle/20250215_CIBMTR/death_rate.csv"
  # 現在の日時を取得してフォーマット
  current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
  # ディレクトリ名を作成
  directory_name = f"/content/drive/MyDrive/Kaggle/20250215_CIBMTR/model_{current_datetime}"

  # ディレクトリのパスを指定
  directory_path = Path(directory_name)

  # ディレクトリを作成
  directory_path.mkdir(parents=True, exist_ok=True)

  train_path = Path('/content/drive/MyDrive/Kaggle/20250215_CIBMTR/data/train.csv')
  test_path = Path('/content/drive/MyDrive/Kaggle/20250215_CIBMTR/data/test.csv')
  subm_path = Path('/content/drive/MyDrive/Kaggle/20250215_CIBMTR/data/sample_submission.csv')


elif "KAGGLE_KERNEL_RUN_TYPE" in os.environ:
  !pip install /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
  !pip install /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
  !pip install /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
  !pip install /kaggle/input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
  !pip install /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl




  # CSVファイルのパスを指定
  csv_file_path = "/kaggle/input/world-age-death-rate/death_rate.csv"
  # 現在の日時を取得してフォーマット
  current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
  # ディレクトリ名を作成
  directory_name = f"model_{current_datetime}"

  # ディレクトリのパスを指定
  directory_path = Path(directory_name)

  # ディレクトリを作成
  directory_path.mkdir(parents=True, exist_ok=True)
  print("Kaggle Notebooks で実行中")

  train_path = Path('/kaggle/input/equity-post-HCT-survival-predictions/train.csv')
  test_path = Path('/kaggle/input/equity-post-HCT-survival-predictions/test.csv')
  subm_path = Path('/kaggle/input/equity-post-HCT-survival-predictions/sample_submission.csv')
else:
  print("ローカル環境または他の環境で実行中")

Processing /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
autograd is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
  Preparing metadata (setup.py) ... done
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=a626c0ea2e03cdfbd85019100460755fac5ecd6c2063abe4c069804edafbc7fe
  Stored in directory: /root/.cache/pip/wheels/6b/b5/e0/4c79e15c0b5f2c15ecf613c720bb20daab20a666eb67135155
Successfully built autograd-gamma
  Attempting uninstall: autograd-gamma
    Found existing installation: autograd-gamma 0.5.0
    Uninstalling autograd-gamma-0.5.0:
      Successfully uninstalled autograd-gamma-0.5.0
Processing /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
interface-meta is already installed with the same version as the provided wheel. Use --force-r

In [9]:
from metric import score
from lifelines import CoxPHFitter
from lifelines import KaplanMeierFitter
from lifelines import NelsonAalenFitter

<p style="background-color: rgb(247, 230, 202); font-size: 300%; text-align: center; border-radius: 40px 40px; color: rgb(162, 87, 79); font-weight: bold; font-family: 'Roboto'; border: 4px solid rgb(162, 87, 79);">Configuration</p>

In [10]:
class CFG:

    train_path = Path('/kaggle/input/equity-post-HCT-survival-predictions/train.csv')
    test_path = Path('/kaggle/input/equity-post-HCT-survival-predictions/test.csv')
    subm_path = Path('/kaggle/input/equity-post-HCT-survival-predictions/sample_submission.csv')

    color = '#A2574F'

    batch_size = 32768
    early_stop = 300
    penalizer = 0.01
    n_splits = 5

    weights = [1.0, 1.0, 8.0, 4.0, 8.0, 4.0, 6.0, 6.0]

# loss_function :
# モデルの損失関数を指定します。この例ではRMSE（Root Mean Squared Error）を使っており、回帰問題において予測値の誤差を評価するために用いられます。
# learning_rate :
# 学習率は、モデルが訓練データから学び取る際のステップサイズを決定します。この値が小さければ小さいほど、学習がより慎重に行われるため、収束が遅くなる可能性がありますが、過学習を抑えることができる場合があります。この例では0.03に設定されています。
# random_state :
# ランダムシードの値を設定し、再現性を確保します。この値を固定することで、毎回同じデータ分割や初期状態で実験を行うことができます。この値が42に設定されています。
# task_type :
# モデルの実行に使用する計算リソースのタイプを指定します。この例ではGPUが設定されているため、GPUを使用して計算が行われます。コメントアウトされたCPUは、CPUを使用するオプションです。
# num_trees :
# 学習に使用するツリーの数を指定します。この例では6000に設定されており、多くのツリーを用いることでモデルの表現力を高めることができますが、過学習のリスクも増えるため、適切な早期停止の設定が重要です。
# subsample :
# サンプリング比率を指定します。この値は、各ツリーを訓練する際に使用されるデータの割合を示します。この例では0.85となっており、85%のデータを使って各ツリーを学習することを示します。これにより過学習を抑える効果があります。
# reg_lambda :
# L2正則化項の係数を指定します。正則化は過学習を防ぐための手法の一つで、モデルの重みが過度に大きくならないように制約を与えます。この値が大きいほど、モデルの複雑さが制約されます。この例では8.0に設定されています。
# depth :
# 各決定木の深さを指定します。この値が大きいほど、モデルはより複雑なパターンを学習できますが、過学習のリスクも高まります。この例では8に設定されています。
# bootstrap_type :
# ブートストラップサンプリングの手法を指定します。この例ではBernoulliが選択されています。Bernoulliブートストラップは、各サンプリングごとにデータポイントを独立に選ぶ方法です。他のオプションとしては、No（ブートストラップを使用しない）やMVS（モンテカルロボンディング）などがあります。

    ctb_params_1 = {
        'loss_function': 'RMSE',
        'learning_rate': 0.02,
        'random_state': 42,
        'task_type': 'CPU',
        # 'task_type': 'GPU',
        'num_trees': 10000,
        'subsample': 0.8,
        'reg_lambda': 7.0,
        'depth': 7,
        'max_bin': 255
        # 'bootstrap_type': 'Bernoulli'
    }
    ctb_params_23 = {
        'loss_function': 'RMSE',
        'learning_rate': 0.02,
        'random_state': 42,
        'task_type': 'CPU',
        # 'task_type': 'GPU',
        'num_trees': 10000,
        'subsample': 0.8,
        'reg_lambda': 7.0,
        'depth': 7,
        'max_bin': 255
        # 'bootstrap_type': 'Bernoulli'
    }

    cbt_params_efs = {
        'loss_function': 'RMSE',
        'learning_rate': 0.02,
        'random_state': 42,
        'task_type': 'CPU',
        # 'task_type': 'GPU',
        'num_trees': 10000,
        'subsample': 0.8,
        'reg_lambda': 7.0,
        'depth': 7,
        'max_bin': 255
        # 'bootstrap_type': 'Bernoulli'
    }
#     LightGBMパラメータの説明
# objective :
# モデルの目的を定義します。この例ではregressionと指定されており、回帰問題において連続値の予測を行うことを示しています。用途によってbinary（二値分類）やmulticlass（多クラス分類）なども指定できます。
# min_child_samples :
# 子ノードに必要な最小のデータサンプル数を指定します。この値が大きいほど、ノードが分割されるために必要なサンプル数も増え、過学習を抑制する働きがあります。この例では32に設定されています。
# num_iterations :
# 学習に使用するブースティングのイテレーション回数（決定木の数）を指定します。この例では6000に設定されており、モデルの収束を図るために多くの木を使用しますが、過学習のリスクも考慮する必要があります。
# learning_rate :
# 学習率を指定し、学習の速度を調整します。この値が小さいほど、モデルはより慎重に学習を進めることができ、安定性が増しますが、収束が遅くなる可能性があります。この例では0.03に設定されています。
# extra_trees :
# Trueに設定すると、より多様な木を構築するために、ノードの分割をランダムに行うことができます。これにより、モデルのバリエーションが増え、過学習を抑制する効果があります。
# reg_lambda :
# L2正則化項の係数を指定します。大きい値を設定することで、モデルの複雑度を制限し、過学習を防ぐ助けになります。この例では8.0に設定されています。
# reg_alpha :
# L1正則化項の係数を指定します。L1正則化はモデルの重みをゼロにすることがあり、特徴選択の効果を持つことがあります。この例では0.1に設定されています。
# num_leaves :
# 決定木の最大葉ノード数を指定します。この値が大きいほど、モデルの表現力が増しますが、過学習のリスクも高まります。この例では64に設定されています。
# metric :
# モデルの評価指標を指定します。この例ではrmseが設定されており、モデルの性能を根平均二乗誤差で評価します。
# max_depth :
# 決定木の最大深さを指定します。この値を設定することで、過学習を防ぎ、モデルの構造を制約することができます。この例では8に設定されています。
# device :
# モデルの実行に使用するデバイスを指定します。この例ではgpuが設定されており、GPUを使用して計算を行うことを示しています。コメントアウトされているcpuはCPUを使用するオプションです。
# max_bin :
# 特徴量のビンの最大数を指定します。この値を設定することで、カテゴリカルデータや連続値をどれだけ細かく分割するかを決定します。この例では128に設定されています。
# verbose :
# ログの出力レベルを指定します。-1に設定すると、情報メッセージを出力しないことを示しています。
# seed :
# ランダムシードの値を指定します。この値を固定しておくことで、再現性を確保することができます。この例では42に設定されています。

    lgb_params_efs = {
        'objective': 'regression',
        'min_child_samples': 80,
        'num_iterations': 6000,
        'learning_rate': 0.02,
        'extra_trees': False,
        'reg_lambda': 7.0,
        'reg_alpha': 1.0,
        'num_leaves': 50,
        'metric': 'rmse',
        'max_depth': -1,
        # 'device': 'gpu',
        'device': 'cpu',
        'max_bin': 255,
        'subsample': 0.8,            # データのサンプリング率
        'verbose': -1,
        'seed': 42
    }

    lgb_params_1 = {
        'objective': 'regression',
        'min_child_samples': 80,
        'num_iterations': 6000,
        'learning_rate': 0.02,
        'extra_trees': False,
        'reg_lambda': 7.0,
        'reg_alpha': 1.0,
        'num_leaves': 50,
        'metric': 'rmse',
        'max_depth': -1,
        # 'device': 'gpu',
        'device': 'cpu',
        'max_bin': 255,
        'subsample': 0.8,            # データのサンプリング率
        'verbose': -1,
        'seed': 42
    }

    xgb_params = {
    'objective': 'reg:squarederror',  # 回帰タスク用の損失関数
    'eval_metric': 'rmse',           # 評価指標
    'learning_rate': 0.02,           # 学習率（低めに設定）
    'max_depth': 7,                  # ツリーの深さ
    'subsample': 0.8,                # サンプリング率
    'colsample_bytree': 0.8,         # 特徴量のサンプリング率
    'lambda': 7.0,                   # L2正則化
    'alpha': 0.0,                    # L1正則化（必要に応じて調整）
    'n_estimators': 10000,           # ツリーの本数（早期停止で自動制御）
    'tree_method': 'hist',           # ツリー構築アルゴリズム（CPU用）
    'random_state': 42,              # 再現性のための乱数シード
    # 'early_stopping_rounds': 300     # 早期停止
}

    xgb_params_efs = {
    'objective': 'reg:squarederror',  # 回帰タスク用の損失関数
    'eval_metric': 'rmse',           # 評価指標
    'learning_rate': 0.02,           # 学習率（低めに設定）
    'max_depth': 7,                  # ツリーの深さ
    'subsample': 0.8,                # サンプリング率
    'colsample_bytree': 0.8,         # 特徴量のサンプリング率
    'lambda': 7.0,                   # L2正則化
    'alpha': 0.0,                    # L1正則化（必要に応じて調整）
    'n_estimators': 10000,           # ツリーの本数（早期停止で自動制御）
    'tree_method': 'hist',           # ツリー構築アルゴリズム（CPU用）
    'random_state': 42,              # 再現性のための乱数シード
    # 'early_stopping_rounds': 300     # 早期停止
}

    lgb_params_23 = {
        'objective': 'regression',
        'min_child_samples': 32,
        'num_iterations': 12000,
        'learning_rate': 0.03,
        'extra_trees': True,
        'reg_lambda': 6.0,
        'reg_alpha': 0.1,
        'num_leaves': 64,
        'metric': 'rmse',
        'max_depth': 8,
        # 'device': 'gpu',
        'device': 'cpu',
        'max_bin': 128,
        'verbose': -1,
        'seed': 42
    }

    # Parameters for the first CatBoost model with Cox loss function
    cox1_params = {
        'grow_policy': 'Depthwise',
        'min_child_samples': 8,
        'loss_function': 'Cox',
        'learning_rate': 0.02,
        'random_state': 42,
        # 'task_type': 'GPU',
        'task_type': 'CPU',
        'num_trees': 10000,
        'subsample': 0.8,
        'reg_lambda': 7.0,
        'depth': 7,
        'max_bin': 255
        # 'bootstrap_type': 'Bernoulli'
    }

    # Parameters for the second CatBoost model with Cox loss function
    cox2_params = {
        'grow_policy': 'Lossguide',
        'loss_function': 'Cox',
        'learning_rate': 0.02,
        'random_state': 42,
        'task_type': 'CPU',
        # 'task_type': 'GPU',
        'num_trees': 10000,
        'subsample': 0.8,
        'reg_lambda': 7.0,
        'num_leaves': 32,
        'depth': 7,
        'max_bin': 255
        # 'bootstrap_type': 'Bernoulli'
    }

<p style="background-color: rgb(247, 230, 202); font-size: 300%; text-align: center; border-radius: 40px 40px; color: rgb(162, 87, 79); font-weight: bold; font-family: 'Roboto'; border: 4px solid rgb(162, 87, 79);">Feature Engineering</p>

In [11]:
class FE:

    def __init__(self, batch_size):
        self._batch_size = batch_size

    def load_data(self, path):
#  batch_size
# データをどの程度の大きさで分割して読み込むかを指定。
# 大規模データを扱う際に、メモリ消費量を制御するために使用
        return pl.read_csv(path, batch_size=self._batch_size)

    def recalculate_hla_sums(self, df):
        # Polarsライブラリを使用してデータフレーム（df）に新しい列を追加します。各新しい列は、特定のHLAマッチング列の合計値を計算し、それらの欠損値（Null）はゼロで補完

        df = df.with_columns(
            # hla_match_a_low, hla_match_b_low, hla_match_drb1_highの各列の値（欠損値を0で補完）を合計。
            (pl.col("hla_match_a_low").fill_null(0) + pl.col("hla_match_b_low").fill_null(0) +
             pl.col("hla_match_drb1_high").fill_null(0)).alias("hla_nmdp_6"),

            (pl.col("hla_match_a_low").fill_null(0) + pl.col("hla_match_b_low").fill_null(0) +
             pl.col("hla_match_drb1_low").fill_null(0)).alias("hla_low_res_6"),

            (pl.col("hla_match_a_high").fill_null(0) + pl.col("hla_match_b_high").fill_null(0) +
             pl.col("hla_match_drb1_high").fill_null(0)).alias("hla_high_res_6"),

            (pl.col("hla_match_a_low").fill_null(0) + pl.col("hla_match_b_low").fill_null(0) +
             pl.col("hla_match_c_low").fill_null(0) + pl.col("hla_match_drb1_low").fill_null(0)
            ).alias("hla_low_res_8"),

            (pl.col("hla_match_a_high").fill_null(0) + pl.col("hla_match_b_high").fill_null(0) +
             pl.col("hla_match_c_high").fill_null(0) + pl.col("hla_match_drb1_high").fill_null(0)
            ).alias("hla_high_res_8"),

            (pl.col("hla_match_a_low").fill_null(0) + pl.col("hla_match_b_low").fill_null(0) +
             pl.col("hla_match_c_low").fill_null(0) + pl.col("hla_match_drb1_low").fill_null(0) +
             pl.col("hla_match_dqb1_low").fill_null(0)).alias("hla_low_res_10"),

            (pl.col("hla_match_a_high").fill_null(0) + pl.col("hla_match_b_high").fill_null(0) +
             pl.col("hla_match_c_high").fill_null(0) + pl.col("hla_match_drb1_high").fill_null(0) +
             pl.col("hla_match_dqb1_high").fill_null(0)).alias("hla_high_res_10"),

             (pl.col("diabetes").cast(pl.Utf8) + "_" + pl.col("obesity").cast(pl.Utf8)).alias("diabetes_obesity").cast(pl.Utf8),
            (pl.col("donor_age") - pl.col("age_at_hct")).alias("age_diff_donor-reci"),
            (pl.col("donor_age") - pl.col("age_at_hct")).alias("age_rate_donor-reci"),
            (pl.col("year_hct") - 2000).alias("year_hct")
        )

        return df

    def cast_datatypes(self, df):

        num_cols = [
            'hla_high_res_8',
            'hla_low_res_8',
            'hla_high_res_6',
            'hla_low_res_6',
            'hla_high_res_10',
            'hla_low_res_10',
            'hla_match_dqb1_high',
            'hla_match_dqb1_low',
            'hla_match_drb1_high',
            'hla_match_drb1_low',
            'hla_nmdp_6',
            'year_hct',
            'hla_match_a_high',
            'hla_match_a_low',
            'hla_match_b_high',
            'hla_match_b_low',
            'hla_match_c_high',
            'hla_match_c_low',
            'donor_age',
            'age_at_hct',
            'comorbidity_score',
            'karnofsky_score',
            'efs',
            'efs_time',
            "age_rate_donor-reci",
            "year_hct",
            "age_diff_donor-reci",   
        ]

        for col in df.columns:

            if col in num_cols:
                df = df.with_columns(pl.col(col).fill_null(-1).cast(pl.Float32))

            else:
                df = df.with_columns(pl.col(col).fill_null('Unknown').cast(pl.String))

        return df.with_columns(pl.col('ID').cast(pl.Int32))

    def info(self, df):

        print(f'\nShape of dataframe: {df.shape}')

        mem = df.memory_usage().sum() / 1024**2
        print('Memory usage: {:.2f} MB\n'.format(mem))

        display(df.head())

    def apply_fe(self, path):

        df = self.load_data(path)
        df = self.recalculate_hla_sums(df)
        df = self.cast_datatypes(df)
        df = df.to_pandas()

        self.info(df)

        cat_cols = [col for col in df.columns if df[col].dtype == pl.String]

        return df, cat_cols

In [12]:
fe = FE(CFG.batch_size)

In [13]:
train_data, cat_cols = fe.apply_fe(train_path)


Shape of dataframe: (28800, 63)
Memory usage: 10.88 MB



,ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,graft_type,vent_hist,renal_issue,pulm_severe,prim_disease_hct,hla_high_res_6,cmv_status,hla_high_res_10,hla_match_dqb1_high,tce_imm_match,hla_nmdp_6,hla_match_c_low,rituximab,hla_match_drb1_low,hla_match_dqb1_low,prod_type,cyto_score_detail,conditioning_intensity,ethnicity,year_hct,obesity,mrd_hct,in_vivo_tcd,tce_match,hla_match_a_high,hepatic_severe,donor_age,prior_tumor,hla_match_b_low,peptic_ulcer,age_at_hct,hla_match_a_low,gvhd_proph,rheum_issue,sex_match,hla_match_b_high,race_group,comorbidity_score,karnofsky_score,hepatic_mild,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,efs,efs_time,diabetes_obesity,age_diff_donor-reci,age_rate_donor-reci
0,0,N/A - non-malignant indication,No,Unknown,No,-1.0,6.0,No TBI,No,6.0,Bone marrow,No,No,No,IEA,6.0,+/+,8.0,2.0,Unknown,6.0,2.0,No,2.0,2.0,BM,Unknown,Unknown,Not Hispanic or Latino,16.0,No,Unknown,Yes,Unknown,2.0,No,-1.000000,No,2.0,No,9.942000,2.0,FKalone,No,M-F,2.0,More than one race,0.0,90.0,No,Unknown,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,42.355999,No_No,-1.000000,-1.000000
1,1,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,Peripheral blood,No,No,No,AML,6.0,+/+,10.0,2.0,P/P,6.0,2.0,No,2.0,2.0,PB,Intermediate,MAC,Not Hispanic or Latino,8.0,No,Positive,No,Permissive,2.0,No,72.290001,No,2.0,No,43.705002,2.0,Other GVHD Prophylaxis,No,F-F,2.0,Asian,3.0,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0,1.0,4.672000,No_No,28.584999,28.584999
2,2,N/A - non-malignant indication,No,Unknown,No,2.0,8.0,No TBI,No,6.0,Bone marrow,No,No,No,HIS,6.0,+/+,10.0,2.0,P/P,6.0,2.0,No,2.0,2.0,BM,Unknown,Unknown,Not Hispanic or Latino,19.0,No,Unknown,Yes,Unknown,2.0,No,-1.000000,No,2.0,No,33.997002,2.0,Cyclophosphamide alone,No,F-M,2.0,More than one race,0.0,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,19.792999,No_No,-1.000000,-1.000000
3,3,High,No,Intermediate,No,2.0,8.0,No TBI,No,6.0,Bone marrow,No,No,No,ALL,6.0,+/+,10.0,2.0,P/P,6.0,2.0,No,2.0,2.0,BM,Intermediate,MAC,Not Hispanic or Latino,9.0,No,Positive,No,Permissive,2.0,No,29.230000,No,2.0,No,43.244999,2.0,FK+ MMF +- others,No,M-M,2.0,White,0.0,90.0,Yes,Permissive mismatched,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,102.348999,No_No,-14.015000,-14.015000
4,4,High,No,Unknown,No,2.0,8.0,No TBI,No,6.0,Peripheral blood,No,No,No,MPN,6.0,+/+,10.0,2.0,Unknown,6.0,2.0,No,2.0,2.0,PB,Unknown,MAC,Hispanic or Latino,18.0,No,Unknown,Yes,Unknown,2.0,No,56.810001,No,2.0,No,29.740000,2.0,TDEPLETION +- other,No,M-F,2.0,American Indian or Alaska Native,1.0,90.0,No,Permissive mismatched,Related,MEL,8.0,No,2.0,No,10.0,0.0,16.223000,No_No,27.070000,27.070000


In [14]:
train_data["diabetes_obesity"].fillna("NONE").value_counts()

diabetes_obesity
No_No                20490
Yes_No                3761
Unknown               2621
No_Yes                1213
Yes_Yes                467
Not done_No            122
No_Not done             84
Yes_Not done            27
Not done_Yes            14
Not done_Not done        1
Name: count, dtype: int64

In [15]:
test_data, _ = fe.apply_fe(test_path)


Shape of dataframe: (3, 61)
Memory usage: 0.00 MB



,ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,graft_type,vent_hist,renal_issue,pulm_severe,prim_disease_hct,hla_high_res_6,cmv_status,hla_high_res_10,hla_match_dqb1_high,tce_imm_match,hla_nmdp_6,hla_match_c_low,rituximab,hla_match_drb1_low,hla_match_dqb1_low,prod_type,cyto_score_detail,conditioning_intensity,ethnicity,year_hct,obesity,mrd_hct,in_vivo_tcd,tce_match,hla_match_a_high,hepatic_severe,donor_age,prior_tumor,hla_match_b_low,peptic_ulcer,age_at_hct,hla_match_a_low,gvhd_proph,rheum_issue,sex_match,hla_match_b_high,race_group,comorbidity_score,karnofsky_score,hepatic_mild,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,diabetes_obesity,age_diff_donor-reci,age_rate_donor-reci
0,28800,N/A - non-malignant indication,No,Unknown,No,-1.0,6.0,No TBI,No,6.0,Bone marrow,No,No,No,IEA,6.0,+/+,8.0,2.0,Unknown,6.0,2.0,No,2.0,2.0,BM,Unknown,Unknown,Not Hispanic or Latino,16.0,No,Unknown,Yes,Unknown,2.0,No,-1.000000,No,2.0,No,9.942000,2.0,FKalone,No,M-F,2.0,More than one race,0.0,90.0,No,Unknown,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,No_No,-1.000000,-1.000000
1,28801,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,Peripheral blood,No,No,No,AML,6.0,+/+,10.0,2.0,P/P,6.0,2.0,No,2.0,2.0,PB,Intermediate,MAC,Not Hispanic or Latino,8.0,No,Positive,No,Permissive,2.0,No,72.290001,No,2.0,No,43.705002,2.0,Other GVHD Prophylaxis,No,F-F,2.0,Asian,3.0,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0,No_No,28.584999,28.584999
2,28802,N/A - non-malignant indication,No,Unknown,No,2.0,8.0,No TBI,No,6.0,Bone marrow,No,No,No,HIS,6.0,+/+,10.0,2.0,P/P,6.0,2.0,No,2.0,2.0,BM,Unknown,Unknown,Not Hispanic or Latino,19.0,No,Unknown,Yes,Unknown,2.0,No,-1.000000,No,2.0,No,33.997002,2.0,Cyclophosphamide alone,No,F-M,2.0,More than one race,0.0,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0,No_No,-1.000000,-1.000000


In [16]:
# CSVファイルを読み込む
death_rate_df = pd.read_csv(csv_file_path)

# "男平均" と "女平均" の行ごとの平均を計算して新しいカラム "全平均" に追加
death_rate_df['全平均'] = death_rate_df[['男平均', '女平均']].mean(axis=1)

# '男平均', '女平均', '全平均'の累積和カラムを作成
death_rate_df['男平均_累積和'] = death_rate_df['男平均'].cumsum()
death_rate_df['女平均_累積和'] = death_rate_df['女平均'].cumsum()
death_rate_df['全平均_累積和'] = death_rate_df['全平均'].cumsum()

# データフレームを表示
display(death_rate_df)

,年 齢,男平均,女平均,全平均,男平均_累積和,女平均_累積和,全平均_累積和
0,0,7.100,5.975,6.5375,7.100,5.975,6.5375
1,1～4,0.350,0.300,0.3250,7.450,6.275,6.8625
2,5～9,0.175,0.125,0.1500,7.625,6.400,7.0125
3,10～14,0.200,0.150,0.1750,7.825,6.550,7.1875
4,15～19,0.650,0.250,0.4500,8.475,6.800,7.6375
5,20～24,1.050,0.325,0.6875,9.525,7.125,8.3250
6,25～29,1.325,0.425,0.8750,10.850,7.550,9.2000
7,30～34,1.550,0.625,1.0875,12.400,8.175,10.2875
8,35～39,2.100,0.875,1.4875,14.500,9.050,11.7750
9,40～44,3.125,1.400,2.2625,17.625,10.450,14.0375


In [17]:

# =====================
# 1) 年齢区分を返す関数
# =====================
def get_age_group(age: float) -> str:
    """
    death_rate_dfの「年齢」カラムに合わせて、数値ageを区間ラベルに変換する。
    ただし age == -1 は不明を示すため None を返す。
    """
    if age == -1:
        return None

    # 0歳
    if age == 0:
        return "0"
    # 1～4歳
    elif 1 <= age < 5:
        return " 1～4"
    elif 5 <= age < 10:
        return " 5～9"
    elif 10 <= age < 15:
        return "10～14"
    elif 15 <= age < 20:
        return "15～19"
    elif 20 <= age < 25:
        return "20～24"
    elif 25 <= age < 30:
        return "25～29"
    elif 30 <= age < 35:
        return "30～34"
    elif 35 <= age < 40:
        return "35～39"
    elif 40 <= age < 45:
        return "40～44"
    elif 45 <= age < 50:
        return "45～49"
    elif 50 <= age < 55:
        return "50～54"
    elif 55 <= age < 60:
        return "55～59"
    elif 60 <= age < 65:
        return "60～64"
    elif 65 <= age < 70:
        return "65～69"
    elif 70 <= age < 75:
        return "70～74"
    elif 75 <= age < 80:
        return "75～79"
    elif 80 <= age < 85:
        return "80～84"
    elif 85 <= age < 90:
        return "85～89"
    elif 90 <= age < 95:
        return "90～94"
    elif 95 <= age < 100:
        return "95～99"
    else:
        # 100歳以上
        return "100歳以上"


# =====================
# 2) sex_match から性別を取り出す関数
# =====================
def get_donor_sex(sex_match: str) -> str:
    """
    sex_match: "M-F" のような文字列
       → ドナーの性別(M/F)を返す
    sex_matchがNaNなど不正ならNoneを返す(= 性別不明)
    """
    if pd.isnull(sex_match):
        return None
    parts = sex_match.split("-")
    if len(parts) == 2:
        return parts[0].strip()  # ドナー側
    return None

def get_recipient_sex(sex_match: str) -> str:
    """
    sex_match: "M-F" のような文字列
       → レシピエントの性別(M/F)を返す
    sex_matchがNaNなど不正ならNoneを返す(= 性別不明)
    """
    if pd.isnull(sex_match):
        return None
    parts = sex_match.split("-")
    if len(parts) == 2:
        return parts[1].strip()  # レシピエント側
    return None


# =====================
# 3) death_rate_df を辞書化: { age_label: {"M": male, "F": female, "All": overall}, ... }
# =====================
death_rate_dict = {}
for _, row in death_rate_df.iterrows():
    age_label = row["年  齢"]        # "0", "1～4", "5～9", ... "100歳以上"
    male_rate = row["男平均"]     # float
    female_rate = row["女平均"]   # float
    all_rate   = row["全平均"]    # float
    cum_male_rate = row["男平均_累積和"]     # float
    cum_female_rate = row["女平均_累積和"]   # float
    cum_all_rate   = row["全平均_累積和"]    # float
    death_rate_dict[age_label] = {
        "M": (male_rate, cum_male_rate),
        "F": (female_rate, cum_female_rate),
        "All": (all_rate, cum_all_rate)
    }


# =====================
# 4) (年齢, 性別) から死亡率を取得する関数
# =====================
def get_death_rate(age: float, sex: str) -> float:
    """
    入力:
      age: 年齢 (ドナー or レシピエントの年齢)
      sex: 'M' または 'F'、またはNone (不明)
    戻り値:
      death_rate: floatまたは np.nan
    """
    # まず「ageが-1」(不明)なら np.nanを返す
    if age == -1:
        return np.nan

    # 年齢区分を取得
    age_label = get_age_group(age)
    if (age_label is None) or (age_label not in death_rate_dict):
        # 万が一該当しない場合は np.nan でも良いが、
        # ここでは "100歳以上" の全平均を返す等、運用次第で対応を変えてください。
        return np.nan

    # 性別が不明(None)なら全平均を返す
    if sex is None:
        return death_rate_dict[age_label]["All"][0]

    # 性別が "M" または "F" なら対応する死亡率を返す
    if sex == "M":
        return death_rate_dict[age_label]["M"][0]
    elif sex == "F":
        return death_rate_dict[age_label]["F"][0]
    else:
        # 想定外の文字が入っている場合も全平均にフォールバック (例)
        return death_rate_dict[age_label]["All"][0]

# =====================
# 4) (年齢, 性別) から累積死亡率を取得する関数
# =====================
def get_cum_death_rate(age: float, sex: str) -> float:
    """
    入力:
      age: 年齢 (ドナー or レシピエントの年齢)
      sex: 'M' または 'F'、またはNone (不明)
    戻り値:
      death_rate: floatまたは np.nan
    """
    # まず「ageが-1」(不明)なら np.nanを返す
    if age == -1:
        return np.nan

    # 年齢区分を取得
    age_label = get_age_group(age)
    if (age_label is None) or (age_label not in death_rate_dict):
        # 万が一該当しない場合は np.nan でも良いが、
        # ここでは "100歳以上" の全平均を返す等、運用次第で対応を変えてください。
        return np.nan

    # 性別が不明(None)なら全平均を返す
    if sex is None:
        return death_rate_dict[age_label]["All"][1]

    # 性別が "M" または "F" なら対応する死亡率を返す
    if sex == "M":
        return death_rate_dict[age_label]["M"][1]
    elif sex == "F":
        return death_rate_dict[age_label]["F"][1]
    else:
        # 想定外の文字が入っている場合も全平均にフォールバック (例)
        return death_rate_dict[age_label]["All"][1]


# =====================
# 5) train_data, test_data に新カラムを追加
# =====================
# ドナーの死亡率
train_data["donor_age_death_rate"] = train_data.apply(
    lambda row: get_death_rate(row["donor_age"], get_donor_sex(row["sex_match"])), axis=1
)

train_data["cum_donor_age_death_rate"] = train_data.apply(
    lambda row: get_cum_death_rate(row["donor_age"], get_donor_sex(row["sex_match"])), axis=1
)
# レシピエントの死亡率
train_data["recipient_age_death_rate"] = train_data.apply(
    lambda row: get_death_rate(row["age_at_hct"], get_recipient_sex(row["sex_match"])), axis=1
)

train_data["cum_recipient_age_death_rate"] = train_data.apply(
    lambda row: get_cum_death_rate(row["age_at_hct"], get_recipient_sex(row["sex_match"])), axis=1
)

# test_data も同様
test_data["donor_age_death_rate"] = test_data.apply(
    lambda row: get_death_rate(row["donor_age"], get_donor_sex(row["sex_match"])), axis=1
)

test_data["cum_donor_age_death_rate"] = test_data.apply(
    lambda row: get_cum_death_rate(row["donor_age"], get_donor_sex(row["sex_match"])), axis=1)


test_data["recipient_age_death_rate"] = test_data.apply(
    lambda row: get_death_rate(row["age_at_hct"], get_recipient_sex(row["sex_match"])), axis=1
)

test_data["cum_recipient_age_death_rate"] = test_data.apply(
    lambda row: get_cum_death_rate(row["age_at_hct"], get_recipient_sex(row["sex_match"])), axis=1
)

# # 新しい特徴量を作成して欠損値を判定
# train_data['recipient_age_death_rate_is_na'] = train_data['recipient_age_death_rate'].isna().astype(int)
# train_data['donor_age_death_rate_is_na'] = train_data['donor_age_death_rate'].isna().astype(int)

# test_data['recipient_age_death_rate_is_na'] = test_data['recipient_age_death_rate'].isna().astype(int)
# test_data['donor_age_death_rate_is_na'] = test_data['donor_age_death_rate'].isna().astype(int)

# recipient_age_death_rate の NaN を同じカラムの NaN 以外の平均値で埋める
train_data['recipient_age_death_rate'] = train_data['recipient_age_death_rate'].fillna(train_data['recipient_age_death_rate'].mean())

# donor_age_death_rate の NaN を同じカラムの NaN 以外の平均値で埋める
train_data['donor_age_death_rate'] = train_data['donor_age_death_rate'].fillna(train_data['donor_age_death_rate'].mean())

# recipient_age_death_rate の NaN を同じカラムの NaN 以外の平均値で埋める
test_data['recipient_age_death_rate'] = test_data['recipient_age_death_rate'].fillna(test_data['recipient_age_death_rate'].mean())

# donor_age_death_rate の NaN を同じカラムの NaN 以外の平均値で埋める
test_data['donor_age_death_rate'] = test_data['donor_age_death_rate'].fillna(test_data['donor_age_death_rate'].mean())

# recipient_age_death_rate の NaN を同じカラムの NaN 以外の平均値で埋める
train_data['cum_recipient_age_death_rate'] = train_data['cum_recipient_age_death_rate'].fillna(train_data['recipient_age_death_rate'].mean())

# donor_age_death_rate の NaN を同じカラムの NaN 以外の平均値で埋める
train_data['cum_donor_age_death_rate'] = train_data['cum_donor_age_death_rate'].fillna(train_data['donor_age_death_rate'].mean())

# recipient_age_death_rate の NaN を同じカラムの NaN 以外の平均値で埋める
test_data['cum_recipient_age_death_rate'] = test_data['cum_recipient_age_death_rate'].fillna(test_data['recipient_age_death_rate'].mean())

# donor_age_death_rate の NaN を同じカラムの NaN 以外の平均値で埋める
test_data['cum_donor_age_death_rate'] = test_data['cum_donor_age_death_rate'].fillna(test_data['donor_age_death_rate'].mean())

In [18]:
display(train_data.describe())

,ID,hla_match_c_high,hla_high_res_8,hla_low_res_6,hla_high_res_6,hla_high_res_10,hla_match_dqb1_high,hla_nmdp_6,hla_match_c_low,hla_match_drb1_low,hla_match_dqb1_low,year_hct,hla_match_a_high,donor_age,hla_match_b_low,age_at_hct,hla_match_a_low,hla_match_b_high,comorbidity_score,karnofsky_score,hla_low_res_8,hla_match_drb1_high,hla_low_res_10,efs,efs_time,age_diff_donor-reci,age_rate_donor-reci,donor_age_death_rate,cum_donor_age_death_rate,recipient_age_death_rate,cum_recipient_age_death_rate
count,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000
mean,14399.500000,1.321042,5.897570,4.691875,4.416111,7.320903,1.242813,4.642431,1.489687,1.466111,1.369861,15.179444,1.299965,39.780029,1.477674,38.663158,1.484271,1.316424,1.657569,81.269447,6.278785,1.392049,7.794271,0.539306,23.237679,2.534599,2.534599,5.714956,22.081755,22.084072,83.971775
std,8313.988213,1.089035,2.581069,1.769359,1.938318,3.153944,1.128069,1.773606,0.915822,0.894156,1.055029,3.153847,1.053784,18.149313,0.887930,21.147446,0.866572,1.035896,2.007735,18.131237,2.327928,0.970314,2.858779,0.498514,24.799696,23.103836,23.103836,7.963808,23.489441,69.441737,260.049517
min,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,-1.000000,-1.000000,-1.000000,8.000000,-1.000000,-1.000000,-1.000000,0.044000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,0.333000,-52.888000,-52.888000,0.250000,5.714956,0.125000,6.275000
25%,7199.750000,1.000000,4.000000,3.000000,3.000000,5.000000,1.000000,3.000000,1.000000,1.000000,1.000000,13.000000,1.000000,26.898001,1.000000,19.539000,1.000000,1.000000,0.000000,70.000000,4.000000,1.000000,6.000000,0.000000,5.619750,-12.843500,-12.843500,1.050000,9.050000,0.650000,8.475000
50%,14399.500000,2.000000,7.000000,6.000000,5.000000,9.000000,2.000000,6.000000,2.000000,2.000000,2.000000,16.000000,2.000000,38.241001,2.000000,41.006001,2.000000,2.000000,1.000000,90.000000,8.000000,2.000000,9.000000,1.000000,9.796500,0.388000,0.388000,2.100000,12.400000,3.125000,16.400000
75%,21599.250000,2.000000,8.000000,6.000000,6.000000,10.000000,2.000000,6.000000,2.000000,2.000000,2.000000,18.000000,2.000000,55.116001,2.000000,55.965250,2.000000,2.000000,2.000000,90.000000,8.000000,2.000000,10.000000,1.000000,35.099998,17.679501,17.679501,6.000000,22.600000,9.525000,31.925000
max,28799.000000,2.000000,8.000000,6.000000,6.000000,10.000000,2.000000,6.000000,2.000000,2.000000,2.000000,20.000000,2.000000,84.800003,2.000000,73.725998,2.000000,2.000000,10.000000,100.000000,8.000000,2.000000,10.000000,1.000000,156.819000,80.768997,80.768997,101.100000,295.625000,327.025000,1324.525000


In [19]:
display(test_data.dtypes)

ID                                int32
dri_score                        object
psych_disturb                    object
cyto_score                       object
diabetes                         object
hla_match_c_high                float32
hla_high_res_8                  float32
tbi_status                       object
arrhythmia                       object
hla_low_res_6                   float32
graft_type                       object
vent_hist                        object
renal_issue                      object
pulm_severe                      object
prim_disease_hct                 object
hla_high_res_6                  float32
cmv_status                       object
hla_high_res_10                 float32
hla_match_dqb1_high             float32
tce_imm_match                    object
hla_nmdp_6                      float32
hla_match_c_low                 float32
rituximab                        object
hla_match_drb1_low              float32
hla_match_dqb1_low              float32


In [20]:
display(train_data.dtypes)

ID                                int32
dri_score                        object
psych_disturb                    object
cyto_score                       object
diabetes                         object
hla_match_c_high                float32
hla_high_res_8                  float32
tbi_status                       object
arrhythmia                       object
hla_low_res_6                   float32
graft_type                       object
vent_hist                        object
renal_issue                      object
pulm_severe                      object
prim_disease_hct                 object
hla_high_res_6                  float32
cmv_status                       object
hla_high_res_10                 float32
hla_match_dqb1_high             float32
tce_imm_match                    object
hla_nmdp_6                      float32
hla_match_c_low                 float32
rituximab                        object
hla_match_drb1_low              float32
hla_match_dqb1_low              float32


In [21]:
nan_counts = train_data.isna().sum()
nan_columns = nan_counts[nan_counts > 0]
print("NaNが存在するカラムとその数:")
print(nan_columns)

NaNが存在するカラムとその数:
Series([], dtype: int64)


In [22]:
nan_counts = test_data.isna().sum()
nan_columns = nan_counts[nan_counts > 0]
print("NaNが存在するカラムとその数:")
print(nan_columns)

NaNが存在するカラムとその数:
Series([], dtype: int64)


In [23]:
nan_records = test_data[test_data['donor_age_death_rate'].isna() | test_data['recipient_age_death_rate'].isna()]
# nan_records = train_data[train_data['donor_age_death_rate'].isna() | train_data['recipient_age_death_rate'].isna()]
# "donor_age_death_rate" が NaN のレコードを抽出
# nan_records = train_data[train_data['donor_age_death_rate'].isna()]
# nan_records = test_data[train_data['recipient_age_death_rate'].isna()]
#
# 結果を表示
display(nan_records[["age_at_hct","donor_age","sex_match", "recipient_age_death_rate", "donor_age_death_rate", "cum_recipient_age_death_rate", "cum_donor_age_death_rate"]])

,age_at_hct,donor_age,sex_match,recipient_age_death_rate,donor_age_death_rate,cum_recipient_age_death_rate,cum_donor_age_death_rate


In [24]:
nan_records = train_data[train_data['donor_age_death_rate'].isna() | train_data['recipient_age_death_rate'].isna()|train_data['cum_donor_age_death_rate'].isna() | train_data['cum_recipient_age_death_rate'].isna()]
# nan_records = train_data[train_data['donor_age_death_rate'].isna() | train_data['recipient_age_death_rate'].isna()]
# "donor_age_death_rate" が NaN のレコードを抽出
# nan_records = train_data[train_data['donor_age_death_rate'].isna()]
# nan_records = train_data[train_data['recipient_age_death_rate'].isna()]

# 結果を表示
display(nan_records[["age_at_hct","donor_age","sex_match", "recipient_age_death_rate", "donor_age_death_rate", "cum_recipient_age_death_rate", "cum_donor_age_death_rate"]])


,age_at_hct,donor_age,sex_match,recipient_age_death_rate,donor_age_death_rate,cum_recipient_age_death_rate,cum_donor_age_death_rate


In [25]:
display(train_data[["age_at_hct","donor_age","sex_match", "recipient_age_death_rate", "donor_age_death_rate", "cum_recipient_age_death_rate", "cum_donor_age_death_rate"]].head())

,age_at_hct,donor_age,sex_match,recipient_age_death_rate,donor_age_death_rate,cum_recipient_age_death_rate,cum_donor_age_death_rate
0,9.942000,-1.000000,M-F,0.125,5.714956,6.400,5.714956
1,43.705002,72.290001,F-F,1.400,23.275000,10.450,70.025000
2,33.997002,-1.000000,F-M,1.550,5.714956,12.400,5.714956
3,43.244999,29.230000,M-M,3.125,1.325000,17.625,10.850000
4,29.740000,56.810001,M-F,0.425,12.475000,7.550,42.800000


In [26]:
# print(nan_records["donor_age_death_rate"].unique())

In [27]:
display(train_data[["age_at_hct","donor_age","sex_match", "recipient_age_death_rate", "donor_age_death_rate", "cum_recipient_age_death_rate", "cum_donor_age_death_rate"]].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28800 entries, 0 to 28799
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   age_at_hct                    28800 non-null  float32
 1   donor_age                     28800 non-null  float32
 2   sex_match                     28800 non-null  object 
 3   recipient_age_death_rate      28800 non-null  float64
 4   donor_age_death_rate          28800 non-null  float64
 5   cum_recipient_age_death_rate  28800 non-null  float64
 6   cum_donor_age_death_rate      28800 non-null  float64
dtypes: float32(2), float64(4), object(1)
memory usage: 1.3+ MB


None

In [28]:
# データフレームを表示
# display(death_rate_df)

<p style="background-color: rgb(247, 230, 202); font-size: 300%; text-align: center; border-radius: 40px 40px; color: rgb(162, 87, 79); font-weight: bold; font-family: 'Roboto'; border: 4px solid rgb(162, 87, 79);">Model Development</p>

In [29]:
class EDA:

    def __init__(self, color, data):
        self._color = color
        self.data = data

    def _template(self, fig, title):

        fig.update_layout(
            title=title,
            title_x=0.5,
            plot_bgcolor='rgba(247, 230, 202, 1)',
            paper_bgcolor='rgba(247, 230, 202, 1)',
            font=dict(color=self._color),
            margin=dict(l=72, r=72, t=72, b=72),
            height=720
        )

        return fig


    # Plotly Expressを使用してヒストグラムを作成し、そのレイアウトとトレースをカスタマイズ
    def distribution_plot(self, col, title):

#         データの準備:
# px.histogramを使ってヒストグラムを作成しています。
# self.data: データフレーム。
# x=col: ヒストグラムのX軸に使用する列。
# nbins=100: ビンの数を100に設定。
# color_discrete_sequence=[self._color]: ヒストグラムの色を指定

        fig = px.histogram(
            self.data,
            x=col,
            nbins=100,
            color_discrete_sequence=[self._color]
        )


#         レイアウトの更新:
# fig.update_layoutを使ってヒストグラムのレイアウトをカスタマイズ。
# xaxis_title='Values': X軸のタイトル。
# yaxis_title='Count': Y軸のタイトル。
# bargap=0.1: バー間の隙間を設定。
# xaxis.dict(gridcolor='grey')とyaxis.dict(gridcolor='grey', zerolinecolor='grey'): グリッド線とゼロ線の色をグレーに設定。

        fig.update_layout(
            xaxis_title='Values',
            yaxis_title='Count',
            bargap=0.1,
            xaxis=dict(gridcolor='grey'),
            yaxis=dict(gridcolor='grey', zerolinecolor='grey')
        )


        # ホバー時の情報をカスタマイズ。
# hovertemplate='Value: %{x:.2f}<br>Count: %{y:,}': ホバー時に表示されるテンプレート。
        # 「ホバー時」とは、マウスカーソルをグラフ上の特定のデータポイントに重ねたときのことを指します。この操作中に、通常はそのポイントに関連する追加情報（例として、X軸の値やY軸のカウントなど）がポップアップとして表示されます。この情報はデータを視覚的に確認する際に便利です。
        fig.update_traces(hovertemplate='Value: %{x:.2f}<br>Count: %{y:,}')

#         テンプレートの適用と表示:
# self._template(fig, f'{title}')でテンプレートを適用（詳細はself._templateメソッドによる）。
# fig.show()でプロットを表示。
        fig = self._template(fig, f'{title}')
        fig.show()

    def _plot_cv(self, scores, title, metric='Stratified C-Index'):

        fold_scores = [round(score, 3) for score in scores]
        mean_score = round(np.mean(scores), 3)

        # で新しいグラフを作成。
        fig = go.Figure()

        # 折りたたみスコア（クロスバリデーションの各フォールドのスコア）をプロット。
#         x軸にはフォールド番号（1からスコア数まで）、y軸には該当するスコアを指定。
# モードは'markers'で、プロットはダイヤ型のマーカーを使用。
# hovertemplateでホバー時に「Fold x: score」を表示。
        fig.add_trace(go.Scatter(
            x = list(range(1, len(fold_scores) + 1)),
            y = fold_scores,
            mode = 'markers',
            name = 'Fold Scores',
            marker = dict(size = 27, color=self._color, symbol='diamond'),
            text = [f'{score:.3f}' for score in fold_scores],
            hovertemplate = 'Fold %{x}: %{text}<extra></extra>',
            hoverlabel = dict(font=dict(size=18)) # {'font': {'size': 18}}
        ))


        #         平均線の追加:
# もう一つのトレースを追加し、クロスバリデーションスコアの平均値を示す水平な破線（ダッシュ）をプロット。
# 平均線にはホバー情報が表示されません。
        fig.add_trace(go.Scatter(
            x = [1, len(fold_scores)],
            y = [mean_score, mean_score],
            mode = 'lines',
            name = f'Mean: {mean_score:.3f}',
            line = dict(dash = 'dash', color = '#B22222'),
            hoverinfo = 'none'
        ))


# レイアウトの更新:
# fig.update_layout(...)で全体のレイアウトを調整。
# グラフのタイトルに平均スコアを含め、X軸およびY軸に適切なタイトルを追加。
# 背景色や文字色を設定し、グリッド線やゼロ線をグレーにすることで見やすくします。
# X軸にはフォールドを示す線形タックを設定。
        fig.update_layout(
            title = f'{title} | Cross-validation Mean {metric} Score: {mean_score}',
            xaxis_title = 'Fold',
            yaxis_title = f'{metric} Score',
            plot_bgcolor = 'rgba(247, 230, 202, 1)',
            paper_bgcolor = 'rgba(247, 230, 202, 1)',
            font = dict(color=self._color),
            xaxis = dict(
                gridcolor = 'grey',
                tickmode = 'linear',
                tick0 = 1,
                dtick = 1,
                range = [0.5, len(fold_scores) + 0.5],
                zerolinecolor = 'grey'
            ),

            yaxis = dict(
                gridcolor = 'grey',
                zerolinecolor = 'grey'
            )
        )

        fig.show()

累積ハザード関数の出力は確率そのものではなく、イベントが発生するリスクの累積量です。

### 特徴

1. **リスクの累積量**:
   - 累積ハザード関数は時間の経過に伴うイベント発生の累積リスクを示します。この値が大きいほど、それまでの時間にイベントが発生するリスクが高いことを示しています。

2. **確率との関係**:
   - 累積ハザード関数から生存関数（特定の時間までイベントが発生しない確率）を計算できます。生存関数は、累積ハザード関数を用いて次の式で表されます：

   $$ S(t) = e^{-H(t)} $$

   ここで、\(S(t)\) は生存関数で、\(H(t)\) は累積ハザード関数の値です。このため生存関数は時間とともに減少し、累積したリスクからイベント発生しない確率を導出できます。

したがって、累積ハザード関数そのものは確率ではありませんが、この関数を使って生存確率などの確率的な指標を導出することができます。

In [30]:
class Targets:

    def __init__(self, data, cat_cols, penalizer, n_splits):

        self.data = data
        self.cat_cols = cat_cols

        self._length = len(self.data)
        self._penalizer = penalizer
        self._n_splits = n_splits

    def _prepare_cv(self):

        oof_preds = np.zeros(self._length)

        cv = KFold(n_splits=self._n_splits, shuffle=True, random_state=42)

        return cv, oof_preds

    def validate_model(self, preds, title):

        y_true = self.data[['ID', 'efs', 'efs_time', 'race_group']].copy()
        y_pred = self.data[['ID']].copy()

        y_pred['prediction'] = preds

        c_index_score = score(y_true.copy(), y_pred.copy(), 'ID')
        # print(f'Overall Stratified C-Index Score for {title}: {c_index_score:.4f}')

        return c_index_score

    def create_target1(self, directory_path):

        '''
        Inside the CV loop, constant columns are dropped if they exist in a fold. Otherwise, the code produces error:

        delta contains nan value(s). Convergence halted. Please see the following tips in the lifelines documentation:

        CVループ内では、もしフォールドに定数列が存在すればそれを削除します。そうしないと、以下のようなエラーが発生します：

「deltaにNaN値が含まれています。収束が停止しました。詳細はlifelinesのドキュメントの次のヒントを参照してください。」
        https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-model
        '''

        cv, oof_preds = self._prepare_cv()

        # Apply one hot encoding to categorical columns
        data = pd.get_dummies(self.data, columns=self.cat_cols, drop_first=True).drop('ID', axis=1)

        for fold, (train_index, valid_index) in enumerate(cv.split(data)):

            train_data = data.iloc[train_index]
            valid_data = data.iloc[valid_index]

            # Drop constant columns if they exist
            train_data = train_data.loc[:, train_data.nunique() > 1]
            valid_data = valid_data[train_data.columns]

# CoxPHFitterは、以下の入力を使用して分析を行い、特定の出力を生成します。
# 入力:
# 観察時間 (duration_col):
# 各サンプルのフォローアップ期間や生存時間を示すデータです。
# イベントが発生したかどうか (event_col):
# 各サンプルについて、イベント（例えば死亡や故障）が実際に発生したか、打ち切られたかを示す二値データです（1はイベント発生、0は打ち切り）。
# 共変量:
# ハザードに影響を与えるとされる変数のセットです。例えば、年齢、性別、治療法など。
# 出力:
# ハザード比（係数） :
# 各共変量が生存に与える影響を推定した係数（ログハザード比）を提供します。この係数は、その変数がハザードにどの程度の影響を与えるかを定量化します。
# リスク比の解釈:
# 推定された係数をもとに、共変量がリスクに与える影響（例えば、ある変数が1単位増えるとリスクが何倍になるか）を解釈します。
# 要約統計量と信頼区間:
# モデルの適合度や共変量に関する信頼区間、p値などの詳細な統計情報を提供します。
# 生存関数とハザード関数の予測:
# 新しいデータに対する予測生存関数や基礎ハザード率を計算することができます。
# これにより、CoxPHFitterは、異なる共変量が生存時間にどのような影響を及ぼすかを分析し、リスクを定量的に評価することが可能です。

# 学習時
# --------------------------------------------------------------------------------------
            cph = CoxPHFitter(penalizer=self._penalizer)
            cph.fit(train_data, duration_col='efs_time', event_col='efs')
            # モデルを保存
            joblib.dump(cph, directory_path / f"cph_model_fold{fold}.joblib")
# -----------------------------------------------------------------------------------------
# モデルを読み込むとき
# -----------------------------------------------------------
            # モデルをロード
            # cph = joblib.load(f"cph_model_fold{fold}.joblib")
# --------------------------------------------------------------------------------

            oof_preds[valid_index] = cph.predict_partial_hazard(valid_data)

        self.data['target1'] = oof_preds
        self.validate_model(oof_preds, 'Cox')

        return self.data

    def create_target2(self, directory_path):

        cv, oof_preds = self._prepare_cv()

        for fold, (train_index, valid_index) in enumerate(cv.split(self.data)):

            train_data = self.data.iloc[train_index]
            valid_data = self.data.iloc[valid_index]

# KaplanMeierFitterを使用すると、特定の時間における生存率を変数として取得することができます。
            # 学習時
# --------------------------------------------------------------------------------------
            kmf = KaplanMeierFitter()
            kmf.fit(durations=train_data['efs_time'], event_observed=train_data['efs'])

            # モデルを保存
            joblib.dump(kmf, directory_path / f"kmf_model_fold{fold}.joblib")
# -----------------------------------------------------------------------------------------
# モデルを読み込むとき
# -----------------------------------------------------------
            # モデルをロード
            # kmf = joblib.load(f"kmf_model_fold{fold}.joblib")
# --------------------------------------------------------------------------------

            # 検証データ（valid_data）の特定の時間における生存率を予測し、それを配列oof_predsのvalid_index位置に格納
            oof_preds[valid_index] = kmf.survival_function_at_times(valid_data['efs_time']).values

        self.data['target2'] = oof_preds
        self.validate_model(oof_preds, 'Kaplan-Meier')

        return self.data

    def create_target3(self, directory_path):

        cv, oof_preds = self._prepare_cv()

        for fold, (train_index, valid_index) in enumerate(cv.split(self.data)):

            train_data = self.data.iloc[train_index]
            valid_data = self.data.iloc[valid_index]

                        # 学習時
# --------------------------------------------------------------------------------------
            # Nelson-Aalen累積ハザード推定量を計算するためのクラスのインスタンスを作成
            naf = NelsonAalenFitter()

            naf.fit(durations=train_data['efs_time'], event_observed=train_data['efs'])


            # モデルを保存
            joblib.dump(naf, directory_path / f"naf_model_fold{fold}.joblib")
# -----------------------------------------------------------------------------------------
# モデルを読み込むとき
# -----------------------------------------------------------
            # モデルをロード
            # naf = joblib.load(directory_path / f"naf_model_fold{fold}.joblib")
# --------------------------------------------------------------------------------

            oof_preds[valid_index] = -naf.cumulative_hazard_at_times(valid_data['efs_time']).values

        self.data['target3'] = oof_preds
        self.validate_model(oof_preds, 'Nelson-Aalen')

        return self.data

    def create_target4(self):

        self.data['target4'] = self.data.efs_time.copy()
        self.data.loc[self.data.efs == 0, 'target4'] *= -1

        return self.data

In [31]:
from scipy.stats import boxcox

class MD:

    def __init__(self, color, data, cat_cols, early_stop, penalizer, n_splits):

        self.eda = EDA(color, data)
        self.targets = Targets(data, cat_cols, penalizer, n_splits)

        self.data = data
        self.cat_cols = cat_cols
        self._early_stop = early_stop
        self.efs_lgb_models = []
        self.efs_cbt_models = []
        self.efs_xgb_models = []

    def create_targets(self, lgb_params, cbt_params, xgb_params, directory_path):
        print("target_1開始")
        self.data = self.targets.create_target1(directory_path)
        print("target_1完了")
        self.data = self.targets.create_target2(directory_path)
        print("target_2完了")
        self.data = self.targets.create_target3(directory_path)
        print("target_3完了")
        self.data = self.targets.create_target4()
        print("target_4完了")

        self.data["efs_pred_lgb"] = 0  # 事前にカラムを用意（任意）cat
        self.data["efs_pred_cbt"] = 0  # 事前にカラムを用意（任意）
        self.data["efs_pred_xgb"] = 0  # 事前にカラムを用意（任意）cat
        # epsの予測モデルを追加
        for col in self.cat_cols:
            self.data[col] = self.data[col].astype('category')

        cv, oof_preds = self.targets._prepare_cv()

        print(self.data.columns)
        X = self.data.drop(['ID', 'efs', 'efs_time', 'target1', 'target2', 'target3', 'target4',
                            "efs_pred_lgb", "efs_pred_cbt", "efs_pred_xgb"
                           ], axis=1, errors='ignore')
        y = self.data['efs']
        w =  1 + self.data["efs"]  # 最小1、最大2の重み

        models, fold_scores = [], []

        for fold, (train_index, valid_index) in enumerate(cv.split(X, y)):
            # 訓練データとバリデーションデータの分割
            X_train = X.iloc[train_index]
            X_valid = X.iloc[valid_index]
            y_train = y.iloc[train_index]
            y_valid = y.iloc[valid_index]
            w_train = w.iloc[train_index]

# lgb-------------------------------------------------------------------------------------------------

            model = lgb.LGBMRegressor(**lgb_params)

            model.fit(
                X_train,
                y_train,
                sample_weight=w_train,
                eval_set=[(X_valid, y_valid)],
                eval_metric='rmse',
                callbacks=[lgb.early_stopping(self._early_stop, verbose=1), lgb.log_evaluation(period=1000)]
            )
            # モデルを保存
            joblib.dump(model, directory_path / f'lightgbm_model_efs_fold{fold}.pkl')

            self.efs_lgb_models.append(model)

            oof_preds[valid_index] = model.predict(X_valid)

            self.data.loc[valid_index, "efs_pred_lgb"] = oof_preds[valid_index]

        print("lgb_efs完了")
# catboost-----------------------------------------------------------------------------------------------------
        for fold, (train_index, valid_index) in enumerate(cv.split(X, y)):
            # 訓練データとバリデーションデータの分割
            X_train = X.iloc[train_index]
            X_valid = X.iloc[valid_index]
            y_train = y.iloc[train_index]
            y_valid = y.iloc[valid_index]
            w_train = w.iloc[train_index]
            model = CatBoostRegressor(**cbt_params, verbose=0, cat_features=self.cat_cols)

            model.fit(
                X_train,
                y_train,
                sample_weight=w_train,
                eval_set=(X_valid, y_valid),
                early_stopping_rounds=self._early_stop,
                verbose=1000
            )
            model_file_path = directory_path / f'catboost_model_efs_fold{fold}.cbm'
            # モデルを保存
            model.save_model(model_file_path)
            self.efs_cbt_models.append(model)

            oof_preds[valid_index] = model.predict(X_valid)
            self.data.loc[valid_index, "efs_pred_cbt"] = oof_preds[valid_index]
        print("ctb_efs完了")
# xgb-----------------------------------------------------------------------------------------------
        for fold, (train_index, valid_index) in enumerate(cv.split(X, y)):
            # 訓練データとバリデーションデータの分割
            X_train = X.iloc[train_index]
            X_valid = X.iloc[valid_index]
            y_train = y.iloc[train_index]
            y_valid = y.iloc[valid_index]
            w_train = w.iloc[train_index]
            X_train_xgb = xgb.DMatrix(X_train, label=y_train, weight=w_train, enable_categorical=True)
            X_valid_xgb = xgb.DMatrix(X_valid, label=y_valid, enable_categorical=True)

             # モデルの学習
            model = xgb.train(
                params=xgb_params,
                dtrain=X_train_xgb,
                num_boost_round=10000,                      # 最大イテレーション数
                evals=[(X_train_xgb, 'train'), (X_valid_xgb, 'valid')],  # 学習データと検証データのセット
                early_stopping_rounds=self._early_stop,    # CatBoostのearly_stopping_roundsに相当
                verbose_eval=1000                          # 学習ログの間隔
                )

             # モデルの保存
            model_file_path = directory_path / f'xgboost_model_efs_fold{fold}.model'

            # JSON形式で保存
            model.save_model(model_file_path.with_suffix(".json"))

            oof_preds[valid_index] = model.predict(X_valid_xgb)
            self.data.loc[valid_index, "efs_pred_xgb"] = oof_preds[valid_index]

            self.efs_xgb_models.append(model)

            # モデルの読み込み
            # model = xgb.Booster()
            # model.load_model("xgboost_model_target1_fold0.json")
        print("xgb_efs完了")

        return self.data

    def train_model(self, params, target, title, directory_path):

        cv, oof_preds = self.targets._prepare_cv()


        X = self.data.drop(['ID', 'efs', 'efs_time', 'target1', 'target2', 'target3', 'target4'], axis=1)
        y = self.data[target]
        w =  1 + self.data["efs"]  # 最小1、最大2の重み

        models, fold_scores = [], []


        for fold, (train_index, valid_index) in enumerate(cv.split(X, y)):
            # 訓練データとバリデーションデータの分割
            X_train = X.iloc[train_index]
            X_valid = X.iloc[valid_index]
            y_train = y.iloc[train_index]
            y_valid = y.iloc[valid_index]
            w_train = w.iloc[train_index]
            w_valid = w.iloc[valid_index]

            # target_encodingをやる場合
            # # 訓練データとバリデーションデータの分割
            # X_train = X.iloc[train_index].copy()
            # X_valid = X.iloc[valid_index].copy()
            # y_train = y.iloc[train_index]
            # y_valid = y.iloc[valid_index]

            # `target` を一時的に追加してエンコーディング計算用に利用
            # X_train['target'] = y_train

            # # カテゴリ変数のターゲットエンコーディング
            # for col in self.cat_cols:
            #     # 訓練データで目的変数の平均を計算
            #     mapping = X_train.groupby(col)['target'].mean()

            #     # バリデーションデータにエンコーディング値を割り当て
            #     X_valid[col] = X_valid[col].map(mapping).astype(float) # 明示的に float に変換

            #     # 訓練データにもエンコーディング値を割り当て
            #     X_train[col] = X_train[col].map(mapping).astype(float) # 明示的に float に変換

            # # エンコーディング計算後、`target` を削除
            # X_train = X_train.drop(columns=['target'])

            if title.startswith('LightGBM'):

                model = lgb.LGBMRegressor(**params)

                model.fit(
                    X_train,
                    y_train,
                    sample_weight=w_train,
                    eval_set=[(X_valid, y_valid)],
                    eval_metric='rmse',
                    callbacks=[lgb.early_stopping(self._early_stop, verbose=1), lgb.log_evaluation(period=1000)]
                )

                # モデルを保存
                joblib.dump(model, directory_path / f'lightgbm_model_{target}_fold{fold}.pkl')

            elif title.startswith('CatBoost'):

                model = CatBoostRegressor(**params, verbose=0, cat_features=self.cat_cols)

                # target_encodingの場合
                # model = CatBoostRegressor(**params, verbose=0, cat_features=[])

                model.fit(
                    X_train,
                    y_train,
                    sample_weight=w_train,
                    eval_set=(X_valid, y_valid),
                    early_stopping_rounds=self._early_stop,
                    verbose=1000
                )
                model_file_path = directory_path / f'catboost_model_{target}_fold{fold}.cbm'
                if target == 'target4':
                    if params['grow_policy'] == 'Depthwise':
                        model_file_path = directory_path / f'catboost_model_{target}_Cox1_fold{fold}.cbm'
                    else:
                        model_file_path = directory_path / f'catboost_model_{target}_Cox2_fold{fold}.cbm'
                # モデルを保存
                model.save_model(model_file_path)

            elif title.startswith('XGBoost'):
                # データセットの準備
                X_train = xgb.DMatrix(X_train, label=y_train, weight=w_train, enable_categorical=True)
                X_valid = xgb.DMatrix(X_valid, label=y_valid, enable_categorical=True)



                # モデルの学習
                model = xgb.train(
                    params=params,
                    dtrain=X_train,
                    num_boost_round=10000,                      # 最大イテレーション数
                    evals=[(X_train, 'train'), (X_valid, 'valid')],  # 学習データと検証データのセット
                    early_stopping_rounds=self._early_stop,    # CatBoostのearly_stopping_roundsに相当
                    verbose_eval=1000                          # 学習ログの間隔
                )

                # モデルの保存
                model_file_path = directory_path / f'xgboost_model_{target}_fold{fold}.model'

                # JSON形式で保存
                model.save_model(model_file_path.with_suffix(".json"))

                # モデルの読み込み
                # model = xgb.Booster()
                # model.load_model("xgboost_model_target1_fold0.json")



            models.append(model)

            oof_preds[valid_index] = model.predict(X_valid)

            y_true_fold = self.data.iloc[valid_index][['ID', 'efs', 'efs_time', 'race_group']].copy()
            y_pred_fold = self.data.iloc[valid_index][['ID']].copy()

            y_pred_fold['prediction'] = oof_preds[valid_index]

            fold_score = score(y_true_fold, y_pred_fold, 'ID')
            fold_scores.append(fold_score)

        self.eda._plot_cv(fold_scores, title)
        self.targets.validate_model(oof_preds, title)

        return models, oof_preds

    def infer_model(self, data, models, target):

        data = data.drop(['ID'], axis=1)

        for col in self.cat_cols:
            data[col] = data[col].astype('category')

        efs_pred_lgb = np.mean([model.predict(data) for model in self.efs_lgb_models], axis=0)
        efs_pred_cbt = np.mean([model.predict(data) for model in self.efs_cbt_models], axis=0)
        efs_pred_xgb = np.mean([model.predict(xgb.DMatrix(data, enable_categorical=True)) for model in self.efs_xgb_models], axis=0)

        data["efs_pred_lgb"] = efs_pred_lgb
        data["efs_pred_cbt"] = efs_pred_cbt
        data["efs_pred_xgb"] = efs_pred_xgb

        # # カテゴリ変数のターゲットエンコーディング
        # for col in self.cat_cols:
        #     # 訓練データで目的変数の平均を計算
        #     mapping = self.data.groupby(col)[target].mean()

        #     # バリデーションデータにエンコーディング値を割り当て
        #     data[col] = data[col].map(mapping).astype(float) # 明示的に float に変換

        return np.mean([model.predict(data) for model in models], axis=0)

    def infer_model_xgb(self, data, models, target):
        
        for col in self.cat_cols:
            data[col] = data[col].astype('category')
            
        data = data.drop(['ID'], axis=1)
        efs_pred_lgb = np.mean([model.predict(data) for model in self.efs_lgb_models], axis=0)
        efs_pred_cbt = np.mean([model.predict(data) for model in self.efs_cbt_models], axis=0)
        efs_pred_xgb = np.mean([model.predict(xgb.DMatrix(data, enable_categorical=True)) for model in self.efs_xgb_models], axis=0)

        data["efs_pred_lgb"] = efs_pred_lgb
        data["efs_pred_cbt"] = efs_pred_cbt
        data["efs_pred_xgb"] = efs_pred_xgb
        # for col in self.cat_cols:
        #     for fold, encoders in self.fold_encoders[target].items():
        #         lbl = encoders[col]
        #         data[col] = data[col].map(lambda x: lbl.transform([x])[0] if x in lbl.classes_ else -1)
        # テストデータをDMatrixに変換
        dtest = xgb.DMatrix(data, enable_categorical=True)

        return np.mean([model.predict(dtest) for model in models], axis=0)

In [32]:
def reduce_memory_usage(df):
    """データフレームのメモリ使用量を削減する関数"""
    start_mem = df.memory_usage().sum() / 1024**2
    print(f'メモリ使用量: {start_mem:.2f} MB')

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:  # 数値型のみ処理
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                # if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                #     df[col] = df[col].astype(np.float16)
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print(f'メモリ削減後: {end_mem:.2f} MB ({100 - 100 * end_mem / start_mem:.1f}% 減少)')
    return df

train_data = reduce_memory_usage(train_data)
test_data = reduce_memory_usage(test_data)

メモリ使用量: 11.76 MB
メモリ削減後: 11.26 MB (4.2% 減少)
メモリ使用量: 0.00 MB
メモリ削減後: 0.00 MB (3.9% 減少)


In [33]:
md = MD(CFG.color, train_data, cat_cols, CFG.early_stop, CFG.penalizer, CFG.n_splits)

In [34]:
# 現在の日時を取得してフォーマット
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# ディレクトリ名を作成
directory_name = f"model_{current_datetime}"

# ディレクトリのパスを指定
directory_path = Path(directory_name)

# ディレクトリを作成
directory_path.mkdir(parents=True, exist_ok=True)

In [35]:
train_data = md.create_targets(CFG.lgb_params_efs, CFG.cbt_params_efs, CFG.xgb_params_efs, directory_path)

target_1開始
target_1完了
target_2完了
target_3完了
target_4完了
Index(['ID', 'dri_score', 'psych_disturb', 'cyto_score', 'diabetes',
       'hla_match_c_high', 'hla_high_res_8', 'tbi_status', 'arrhythmia',
       'hla_low_res_6', 'graft_type', 'vent_hist', 'renal_issue',
       'pulm_severe', 'prim_disease_hct', 'hla_high_res_6', 'cmv_status',
       'hla_high_res_10', 'hla_match_dqb1_high', 'tce_imm_match', 'hla_nmdp_6',
       'hla_match_c_low', 'rituximab', 'hla_match_drb1_low',
       'hla_match_dqb1_low', 'prod_type', 'cyto_score_detail',
       'conditioning_intensity', 'ethnicity', 'year_hct', 'obesity', 'mrd_hct',
       'in_vivo_tcd', 'tce_match', 'hla_match_a_high', 'hepatic_severe',
       'donor_age', 'prior_tumor', 'hla_match_b_low', 'peptic_ulcer',
       'age_at_hct', 'hla_match_a_low', 'gvhd_proph', 'rheum_issue',
       'sex_match', 'hla_match_b_high', 'race_group', 'comorbidity_score',
       'karnofsky_score', 'hepatic_mild', 'tce_div_match', 'donor_related',
       'melphala

In [36]:
# md.data["target1"], _ = yeojohnson(md.data['target1'])
# md.data["target2"], _ = yeojohnson(md.data['target2'])
# md.data["target3"], _ = yeojohnson(md.data['target3'])
# md.data["target4"], _ = yeojohnson(md.data['target4'])

# train_data = md.data


# display(train_data.head)

In [37]:
md.eda.distribution_plot('target1', 'Cox Target')

In [38]:
md.eda.distribution_plot('target2', 'Kaplan-Meier Target')

In [39]:
md.eda.distribution_plot('target3', 'Nelson-Aalen Target')

In [40]:
md.eda.distribution_plot('target4', 'Target for Cox-Loss Models')

In [41]:
fe.info(train_data)


Shape of dataframe: (28800, 74)
Memory usage: 5.78 MB



,ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,graft_type,vent_hist,renal_issue,pulm_severe,prim_disease_hct,hla_high_res_6,cmv_status,hla_high_res_10,hla_match_dqb1_high,tce_imm_match,hla_nmdp_6,hla_match_c_low,rituximab,hla_match_drb1_low,hla_match_dqb1_low,prod_type,cyto_score_detail,conditioning_intensity,ethnicity,year_hct,obesity,mrd_hct,in_vivo_tcd,tce_match,hla_match_a_high,hepatic_severe,donor_age,prior_tumor,hla_match_b_low,peptic_ulcer,age_at_hct,hla_match_a_low,gvhd_proph,rheum_issue,sex_match,hla_match_b_high,race_group,comorbidity_score,karnofsky_score,hepatic_mild,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,efs,efs_time,diabetes_obesity,age_diff_donor-reci,age_rate_donor-reci,donor_age_death_rate,cum_donor_age_death_rate,recipient_age_death_rate,cum_recipient_age_death_rate,target1,target2,target3,target4,efs_pred_lgb,efs_pred_cbt,efs_pred_xgb
0,0,N/A - non-malignant indication,No,Unknown,No,-1.0,6.0,No TBI,No,6.0,Bone marrow,No,No,No,IEA,6.0,+/+,8.0,2.0,Unknown,6.0,2.0,No,2.0,2.0,BM,Unknown,Unknown,Not Hispanic or Latino,16.0,No,Unknown,Yes,Unknown,2.0,No,-1.000000,No,2.0,No,9.942000,2.0,FKalone,No,M-F,2.0,More than one race,0.0,90.0,No,Unknown,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,42.355999,No_No,-1.000000,-1.000000,5.714956,5.714956,0.125,6.400,0.272798,0.461962,-0.772247,-42.355999,0.113302,0.283002,0.159838
1,1,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,Peripheral blood,No,No,No,AML,6.0,+/+,10.0,2.0,P/P,6.0,2.0,No,2.0,2.0,PB,Intermediate,MAC,Not Hispanic or Latino,8.0,No,Positive,No,Permissive,2.0,No,72.290001,No,2.0,No,43.705002,2.0,Other GVHD Prophylaxis,No,F-F,2.0,Asian,3.0,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0,1.0,4.672000,No_No,28.584999,28.584999,23.275000,70.025002,1.400,10.450,0.987582,0.847723,-0.165197,4.672000,0.659746,0.721116,0.679942
2,2,N/A - non-malignant indication,No,Unknown,No,2.0,8.0,No TBI,No,6.0,Bone marrow,No,No,No,HIS,6.0,+/+,10.0,2.0,P/P,6.0,2.0,No,2.0,2.0,BM,Unknown,Unknown,Not Hispanic or Latino,19.0,No,Unknown,Yes,Unknown,2.0,No,-1.000000,No,2.0,No,33.997002,2.0,Cyclophosphamide alone,No,F-M,2.0,More than one race,0.0,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,19.792999,No_No,-1.000000,-1.000000,5.714956,5.714956,1.550,12.400,0.129658,0.460785,-0.774799,-19.792999,-0.021048,-0.031186,0.024396
3,3,High,No,Intermediate,No,2.0,8.0,No TBI,No,6.0,Bone marrow,No,No,No,ALL,6.0,+/+,10.0,2.0,P/P,6.0,2.0,No,2.0,2.0,BM,Intermediate,MAC,Not Hispanic or Latino,9.0,No,Positive,No,Permissive,2.0,No,29.230000,No,2.0,No,43.244999,2.0,FK+ MMF +- others,No,M-M,2.0,White,0.0,90.0,Yes,Permissive mismatched,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,102.348999,No_No,-14.015000,-14.015000,1.325000,10.850000,3.125,17.625,1.340261,0.455588,-0.786139,-102.348999,0.848313,0.789930,0.862152
4,4,High,No,Unknown,No,2.0,8.0,No TBI,No,6.0,Peripheral blood,No,No,No,MPN,6.0,+/+,10.0,2.0,Unknown,6.0,2.0,No,2.0,2.0,PB,Unknown,MAC,Hispanic or Latino,18.0,No,Unknown,Yes,Unknown,2.0,No,56.810001,No,2.0,No,29.740000,2.0,TDEPLETION +- other,No,M-F,2.0,American Indian or Alaska Native,1.0,90.0,No,Permissive mismatched,Related,MEL,8.0,No,2.0,No,10.0,0.0,16.223000,No_No,27.070000,27.070000,12.475000,42.799999,0.425,7.550,0.910374,0.463916,-0.768026,-16.223000,0.514874,0.494372,0.464430


<p style="background-color: rgb(247, 230, 202); font-size: 300%; text-align: center; border-radius: 40px 40px; color: rgb(162, 87, 79); font-weight: bold; font-family: 'Roboto'; border: 4px solid rgb(162, 87, 79);">Models with Cox Target</p>

In [42]:
xgb1_models, xgb1_oof_preds = md.train_model(CFG.xgb_params, target='target1', title='XGBoost', directory_path=directory_path)

[0]	train-rmse:0.89558	valid-rmse:0.87432
[1000]	train-rmse:0.08352	valid-rmse:0.21467
[2000]	train-rmse:0.04620	valid-rmse:0.19847
[3000]	train-rmse:0.03040	valid-rmse:0.19383
[4000]	train-rmse:0.02136	valid-rmse:0.19204
[5000]	train-rmse:0.01551	valid-rmse:0.19119
[6000]	train-rmse:0.01156	valid-rmse:0.19074
[7000]	train-rmse:0.00868	valid-rmse:0.19049
[8000]	train-rmse:0.00659	valid-rmse:0.19032
[9000]	train-rmse:0.00509	valid-rmse:0.19021
[9999]	train-rmse:0.00393	valid-rmse:0.19014
[0]	train-rmse:0.90179	valid-rmse:0.85536
[1000]	train-rmse:0.08292	valid-rmse:0.21248
[2000]	train-rmse:0.04602	valid-rmse:0.19667
[3000]	train-rmse:0.03038	valid-rmse:0.19208
[4000]	train-rmse:0.02124	valid-rmse:0.19037
[5000]	train-rmse:0.01534	valid-rmse:0.18955
[6000]	train-rmse:0.01139	valid-rmse:0.18913
[7000]	train-rmse:0.00854	valid-rmse:0.18892
[8000]	train-rmse:0.00650	valid-rmse:0.18876
[9000]	train-rmse:0.00499	valid-rmse:0.18868
[9999]	train-rmse:0.00386	valid-rmse:0.18861
[0]	train-rmse:0

In [43]:
lgb1_models, lgb1_oof_preds = md.train_model(CFG.lgb_params_1, target='target1', title='LightGBM', directory_path=directory_path)

Training until validation scores don't improve for 300 rounds
[1000]	valid_0's rmse: 0.207734
[2000]	valid_0's rmse: 0.190528
[3000]	valid_0's rmse: 0.185096
[4000]	valid_0's rmse: 0.182553
[5000]	valid_0's rmse: 0.181214
[6000]	valid_0's rmse: 0.180375
Did not meet early stopping. Best iteration is:
[6000]	valid_0's rmse: 0.180375
Training until validation scores don't improve for 300 rounds
[1000]	valid_0's rmse: 0.207801
[2000]	valid_0's rmse: 0.189865
[3000]	valid_0's rmse: 0.184534
[4000]	valid_0's rmse: 0.182281
[5000]	valid_0's rmse: 0.181061
[6000]	valid_0's rmse: 0.180262
Did not meet early stopping. Best iteration is:
[6000]	valid_0's rmse: 0.180262
Training until validation scores don't improve for 300 rounds
[1000]	valid_0's rmse: 0.227557
[2000]	valid_0's rmse: 0.207401
[3000]	valid_0's rmse: 0.201366
[4000]	valid_0's rmse: 0.198604
[5000]	valid_0's rmse: 0.197246
[6000]	valid_0's rmse: 0.196339
Did not meet early stopping. Best iteration is:
[5997]	valid_0's rmse: 0.19633

In [44]:
ctb1_models, ctb1_oof_preds = md.train_model(CFG.ctb_params_1, target='target1', title='CatBoost', directory_path=directory_path)
# 現在のイテレーション数（エポック数）, トレーニングデータに対する損失関数の値, 検証データに対する損失関数の, 検証データでのこれまでの最良の損失関数の値と、それが達成されたイテレーション番号, トレーニングの開始から現在のイテレーションまでに経過した合計時間, 現在の速度でトレーニングが継続した場合に予想される残り時間

0:	learn: 0.8975806	test: 0.8757266	best: 0.8757266 (0)	total: 116ms	remaining: 19m 16s
1000:	learn: 0.2023104	test: 0.2236432	best: 0.2236432 (1000)	total: 1m 53s	remaining: 16m 56s
2000:	learn: 0.1478391	test: 0.1890639	best: 0.1890639 (2000)	total: 3m 47s	remaining: 15m 9s
3000:	learn: 0.1211524	test: 0.1733757	best: 0.1733757 (3000)	total: 5m 42s	remaining: 13m 17s
4000:	learn: 0.1043975	test: 0.1646160	best: 0.1646160 (4000)	total: 7m 38s	remaining: 11m 27s
5000:	learn: 0.0935972	test: 0.1597655	best: 0.1597655 (5000)	total: 9m 35s	remaining: 9m 35s
6000:	learn: 0.0855080	test: 0.1565146	best: 0.1565146 (6000)	total: 11m 32s	remaining: 7m 41s
7000:	learn: 0.0790160	test: 0.1540925	best: 0.1540925 (7000)	total: 13m 30s	remaining: 5m 47s
8000:	learn: 0.0734961	test: 0.1520630	best: 0.1520630 (8000)	total: 15m 29s	remaining: 3m 52s
9000:	learn: 0.0689794	test: 0.1506778	best: 0.1506778 (9000)	total: 17m 26s	remaining: 1m 56s
9999:	learn: 0.0653033	test: 0.1495108	best: 0.1495104 (999

In [45]:
xgb1_preds = md.infer_model_xgb(test_data, xgb1_models, 'target1')

In [46]:
lgb1_preds = md.infer_model(test_data, lgb1_models, 'target1')

In [47]:
ctb1_preds = md.infer_model(test_data, ctb1_models, 'target1')

<p style="background-color: rgb(247, 230, 202); font-size: 300%; text-align: center; border-radius: 40px 40px; color: rgb(162, 87, 79); font-weight: bold; font-family: 'Roboto'; border: 4px solid rgb(162, 87, 79);">Models with Kaplan-Meier Target</p>

In [48]:
xgb2_models, xgb2_oof_preds = md.train_model(CFG.xgb_params, target='target2', title='XGBoost', directory_path=directory_path)

[0]	train-rmse:0.17928	valid-rmse:0.18169
[1000]	train-rmse:0.09624	valid-rmse:0.15762
[2000]	train-rmse:0.06590	valid-rmse:0.15633
[3000]	train-rmse:0.04651	valid-rmse:0.15559
[4000]	train-rmse:0.03340	valid-rmse:0.15514
[5000]	train-rmse:0.02424	valid-rmse:0.15480
[6000]	train-rmse:0.01778	valid-rmse:0.15458
[7000]	train-rmse:0.01317	valid-rmse:0.15444
[8000]	train-rmse:0.00984	valid-rmse:0.15433
[9000]	train-rmse:0.00739	valid-rmse:0.15426
[9999]	train-rmse:0.00559	valid-rmse:0.15421
[0]	train-rmse:0.17903	valid-rmse:0.18072
[1000]	train-rmse:0.09621	valid-rmse:0.15662
[2000]	train-rmse:0.06615	valid-rmse:0.15626
[3000]	train-rmse:0.04644	valid-rmse:0.15611
[4000]	train-rmse:0.03332	valid-rmse:0.15602
[4778]	train-rmse:0.02592	valid-rmse:0.15600
[0]	train-rmse:0.17918	valid-rmse:0.18191
[1000]	train-rmse:0.09655	valid-rmse:0.15838
[2000]	train-rmse:0.06621	valid-rmse:0.15745
[3000]	train-rmse:0.04679	valid-rmse:0.15687
[4000]	train-rmse:0.03355	valid-rmse:0.15643
[5000]	train-rmse:0

In [49]:
ctb2_models, ctb2_oof_preds = md.train_model(CFG.ctb_params_23, target='target2', title='CatBoost', directory_path=directory_path)

0:	learn: 0.1794214	test: 0.1816820	best: 0.1816820 (0)	total: 119ms	remaining: 19m 53s
1000:	learn: 0.1483851	test: 0.1606674	best: 0.1606673 (998)	total: 1m 51s	remaining: 16m 46s
2000:	learn: 0.1418525	test: 0.1598971	best: 0.1598932 (1970)	total: 3m 48s	remaining: 15m 11s
3000:	learn: 0.1366236	test: 0.1594824	best: 0.1594752 (2957)	total: 5m 44s	remaining: 13m 22s
4000:	learn: 0.1320498	test: 0.1592099	best: 0.1592077 (3997)	total: 7m 41s	remaining: 11m 32s
5000:	learn: 0.1278956	test: 0.1589555	best: 0.1589493 (4965)	total: 9m 38s	remaining: 9m 38s
6000:	learn: 0.1237582	test: 0.1587509	best: 0.1587466 (5933)	total: 11m 35s	remaining: 7m 43s
7000:	learn: 0.1195610	test: 0.1585970	best: 0.1585942 (6991)	total: 13m 33s	remaining: 5m 48s
8000:	learn: 0.1156927	test: 0.1584543	best: 0.1584516 (7980)	total: 15m 30s	remaining: 3m 52s
9000:	learn: 0.1119331	test: 0.1582936	best: 0.1582934 (8999)	total: 17m 28s	remaining: 1m 56s
Stopped by overfitting detector  (300 iterations wait)

bes

In [50]:
lgb2_models, lgb2_oof_preds = md.train_model(CFG.lgb_params_23, target='target2', title='LightGBM', directory_path=directory_path)

Training until validation scores don't improve for 300 rounds
[1000]	valid_0's rmse: 0.158791
[2000]	valid_0's rmse: 0.157985
[3000]	valid_0's rmse: 0.157415
[4000]	valid_0's rmse: 0.15707
[5000]	valid_0's rmse: 0.156719
[6000]	valid_0's rmse: 0.156444
[7000]	valid_0's rmse: 0.156265
[8000]	valid_0's rmse: 0.15612
[9000]	valid_0's rmse: 0.156011
[10000]	valid_0's rmse: 0.155909
[11000]	valid_0's rmse: 0.155845
[12000]	valid_0's rmse: 0.155777
Did not meet early stopping. Best iteration is:
[11971]	valid_0's rmse: 0.155776
Training until validation scores don't improve for 300 rounds
[1000]	valid_0's rmse: 0.157453
Early stopping, best iteration is:
[1119]	valid_0's rmse: 0.157393
Training until validation scores don't improve for 300 rounds
[1000]	valid_0's rmse: 0.15951
[2000]	valid_0's rmse: 0.158989
[3000]	valid_0's rmse: 0.158537
[4000]	valid_0's rmse: 0.158204
[5000]	valid_0's rmse: 0.157967
[6000]	valid_0's rmse: 0.157706
[7000]	valid_0's rmse: 0.157571
[8000]	valid_0's rmse: 0.1

In [51]:
xgb2_preds = md.infer_model_xgb(test_data, xgb2_models, 'target2')

In [52]:
ctb2_preds = md.infer_model(test_data, ctb2_models, 'target2')

In [53]:
lgb2_preds = md.infer_model(test_data, lgb2_models, 'target2')

<p style="background-color: rgb(247, 230, 202); font-size: 300%; text-align: center; border-radius: 40px 40px; color: rgb(162, 87, 79); font-weight: bold; font-family: 'Roboto'; border: 4px solid rgb(162, 87, 79);">Models with Nelson-Aalen Target</p>

In [54]:
xgb3_models, xgb3_oof_preds = md.train_model(CFG.xgb_params, target='target3', title='XGBoost', directory_path=directory_path)

[0]	train-rmse:0.27177	valid-rmse:0.28146
[1000]	train-rmse:0.14881	valid-rmse:0.24515
[2000]	train-rmse:0.10269	valid-rmse:0.24303
[3000]	train-rmse:0.07292	valid-rmse:0.24178
[4000]	train-rmse:0.05284	valid-rmse:0.24093
[5000]	train-rmse:0.03852	valid-rmse:0.24038
[6000]	train-rmse:0.02836	valid-rmse:0.24002
[7000]	train-rmse:0.02108	valid-rmse:0.23974
[8000]	train-rmse:0.01580	valid-rmse:0.23954
[9000]	train-rmse:0.01188	valid-rmse:0.23942
[9999]	train-rmse:0.00897	valid-rmse:0.23934
[0]	train-rmse:0.27210	valid-rmse:0.27683
[1000]	train-rmse:0.14979	valid-rmse:0.24062
[2000]	train-rmse:0.10337	valid-rmse:0.23996
[2698]	train-rmse:0.08108	valid-rmse:0.23990
[0]	train-rmse:0.27173	valid-rmse:0.28156
[1000]	train-rmse:0.14962	valid-rmse:0.24545
[2000]	train-rmse:0.10331	valid-rmse:0.24377
[3000]	train-rmse:0.07338	valid-rmse:0.24265
[4000]	train-rmse:0.05290	valid-rmse:0.24188
[5000]	train-rmse:0.03842	valid-rmse:0.24136
[6000]	train-rmse:0.02824	valid-rmse:0.24103
[7000]	train-rmse:0

In [55]:
ctb3_models, ctb3_oof_preds = md.train_model(CFG.ctb_params_23, target='target3', title='CatBoost', directory_path=directory_path)

0:	learn: 0.2720131	test: 0.2815051	best: 0.2815051 (0)	total: 122ms	remaining: 20m 20s
1000:	learn: 0.2270863	test: 0.2499214	best: 0.2499214 (1000)	total: 1m 56s	remaining: 17m 25s
2000:	learn: 0.2174083	test: 0.2488968	best: 0.2488968 (2000)	total: 3m 56s	remaining: 15m 44s
3000:	learn: 0.2095026	test: 0.2483453	best: 0.2483453 (3000)	total: 5m 56s	remaining: 13m 51s
4000:	learn: 0.2027515	test: 0.2480471	best: 0.2480399 (3971)	total: 7m 56s	remaining: 11m 53s
5000:	learn: 0.1965011	test: 0.2477498	best: 0.2477496 (4999)	total: 9m 56s	remaining: 9m 55s
6000:	learn: 0.1903286	test: 0.2475336	best: 0.2475312 (5995)	total: 11m 56s	remaining: 7m 57s
7000:	learn: 0.1844991	test: 0.2473085	best: 0.2473021 (6987)	total: 13m 58s	remaining: 5m 59s
8000:	learn: 0.1788490	test: 0.2469528	best: 0.2469407 (7979)	total: 16m	remaining: 3m 59s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.2468704105
bestIteration = 8286

Shrink model to first 8287 iterations.
0:	learn: 0.2723

In [56]:
lgb3_models, lgb3_oof_preds = md.train_model(CFG.lgb_params_23, target='target3', title='LightGBM', directory_path=directory_path)

Training until validation scores don't improve for 300 rounds
[1000]	valid_0's rmse: 0.247005
[2000]	valid_0's rmse: 0.245624
[3000]	valid_0's rmse: 0.244537
[4000]	valid_0's rmse: 0.243831
[5000]	valid_0's rmse: 0.243243
[6000]	valid_0's rmse: 0.242732
[7000]	valid_0's rmse: 0.242407
[8000]	valid_0's rmse: 0.242162
[9000]	valid_0's rmse: 0.241975
[10000]	valid_0's rmse: 0.241824
[11000]	valid_0's rmse: 0.24164
[12000]	valid_0's rmse: 0.241521
Did not meet early stopping. Best iteration is:
[11983]	valid_0's rmse: 0.241518
Training until validation scores don't improve for 300 rounds
[1000]	valid_0's rmse: 0.242665
Early stopping, best iteration is:
[803]	valid_0's rmse: 0.242651
Training until validation scores don't improve for 300 rounds
[1000]	valid_0's rmse: 0.246762
[2000]	valid_0's rmse: 0.245659
[3000]	valid_0's rmse: 0.244789
[4000]	valid_0's rmse: 0.244103
[5000]	valid_0's rmse: 0.243566
[6000]	valid_0's rmse: 0.24329
[7000]	valid_0's rmse: 0.24306
[8000]	valid_0's rmse: 0.24

In [57]:
xgb3_preds = md.infer_model_xgb(test_data, xgb2_models, 'target3')

In [58]:
ctb3_preds = md.infer_model(test_data, ctb3_models, 'target3')

In [59]:
lgb3_preds = md.infer_model(test_data, lgb3_models, 'target3')

<p style="background-color: rgb(247, 230, 202); font-size: 300%; text-align: center; border-radius: 40px 40px; color: rgb(162, 87, 79); font-weight: bold; font-family: 'Roboto'; border: 4px solid rgb(162, 87, 79);">Cox-Loss Models</p>

In [60]:
cox1_models, cox1_oof_preds = md.train_model(CFG.cox1_params, target='target4', title='CatBoost', directory_path=directory_path)

0:	learn: -120740.1631598	test: -25575.0513176	best: -25575.0513176 (0)	total: 88.7ms	remaining: 14m 46s
1000:	learn: -117458.5765939	test: -24943.3290697	best: -24943.1936116 (966)	total: 51.4s	remaining: 7m 41s
Stopped by overfitting detector  (300 iterations wait)

bestTest = -24942.18158
bestIteration = 1195

Shrink model to first 1196 iterations.
0:	learn: -119751.1992176	test: -26412.7827819	best: -26412.7827819 (0)	total: 83.6ms	remaining: 13m 56s
1000:	learn: -116503.6491651	test: -25762.9660290	best: -25762.8982152 (957)	total: 51.4s	remaining: 7m 42s
Stopped by overfitting detector  (300 iterations wait)

bestTest = -25761.6532
bestIteration = 1186

Shrink model to first 1187 iterations.
0:	learn: -120673.6099529	test: -25633.9870813	best: -25633.9870813 (0)	total: 82.6ms	remaining: 13m 45s
Stopped by overfitting detector  (300 iterations wait)

bestTest = -24965.71353
bestIteration = 632

Shrink model to first 633 iterations.
0:	learn: -120751.3453082	test: -25566.9069781	be

In [61]:
cox2_models, cox2_oof_preds = md.train_model(CFG.cox2_params, target='target4', title='CatBoost', directory_path=directory_path)

0:	learn: -120741.3148984	test: -25575.1864528	best: -25575.1864528 (0)	total: 77.8ms	remaining: 12m 57s
1000:	learn: -117660.9165800	test: -24945.8237734	best: -24945.8232443 (998)	total: 1m 13s	remaining: 11m 4s
Stopped by overfitting detector  (300 iterations wait)

bestTest = -24943.82435
bestIteration = 1404

Shrink model to first 1405 iterations.
0:	learn: -119751.4751071	test: -26412.9096357	best: -26412.9096357 (0)	total: 74.5ms	remaining: 12m 24s
1000:	learn: -116715.9414293	test: -25761.5318132	best: -25761.4493325 (996)	total: 1m 14s	remaining: 11m 7s
Stopped by overfitting detector  (300 iterations wait)

bestTest = -25760.27264
bestIteration = 1130

Shrink model to first 1131 iterations.
0:	learn: -120671.2569666	test: -25632.7143960	best: -25632.7143960 (0)	total: 76.4ms	remaining: 12m 43s
1000:	learn: -117606.2973280	test: -24964.1482975	best: -24962.5936608 (757)	total: 1m 13s	remaining: 11m 3s
Stopped by overfitting detector  (300 iterations wait)

bestTest = -24962.59

In [62]:


#  # データフレームに変換
# importance_df = pd.DataFrame(columns = ['Feature', 'Importance'])

# for module_models in models:
#     for model in module_models:

#         # 特徴量の重要度を取得
#         importance = model.feature_importances_
#         features = train_data.drop(['ID', 'efs', 'efs_time', 'target1', 'target2', 'target3', 'target4'], axis=1).columns  # 特徴量の名前を取得

#         # データフレームに変換
#         importance_df_temp = pd.DataFrame({'Feature': features, 'Importance': importance})

#         # 重要度の合計を計算
#         total_importance = importance_df_temp['Importance'].sum()

#         # 正規化を行う
#         importance_df_temp['Normalized_Importance'] = importance_df_temp['Importance'] / total_importance



# # 重要度でソート
# importance_df = importance_df.sort_values(by='Importance', ascending=False)

# # 数値で表示
# print(importance_df)
# # 特徴量の重要度を可視化

#         # plt.figure(figsize=(10, 10))
#         # plt.barh(importance_df['Feature'], importance_df['Importance'], color='skyblue')
#         # plt.xlabel('Importance')
#         # plt.title('Feature Importance')
#         # plt.gca().invert_yaxis()  # 特徴量名が上から表示されるように反転
#         # plt.show()

## 特徴量の重要度

In [63]:
# models = [
#     ctb1_models,
#     lgb1_models,
#     ctb2_models,
#     lgb2_models,
#     ctb3_models,
#     lgb3_models,
#     cox1_models,
#     cox2_models
# ]

# models_name = [
#     "ctb1_models",
#     "lgb1_models",
#     "ctb2_models",
#     "lgb2_models",
#     "ctb3_models",
#     "lgb3_models",
#     "cox1_models",
#     "cox2_models"
# ]



# for module_models, model_name in zip(models, models_name):
#     # 特徴量とその重要度を保存するリストを初期化
#     importance_list = []
#     for model in module_models:

#         # 特徴量の重要度を取得
#         importance = model.feature_importances_
#         features = train_data.drop(['ID', 'efs', 'efs_time', 'target1', 'target2', 'target3', 'target4'], axis=1).columns  # 特徴量の名前を取得

#         # データフレームに変換
#         importance_df_temp = pd.DataFrame({'Feature': features, 'Importance': importance})

#         # 重要度の合計を計算
#         total_importance = importance_df_temp['Importance'].sum()

#         # 正規化を行う
#         importance_df_temp['Normalized_Importance'] = importance_df_temp['Importance'] / total_importance

#         # 正規化された重要度をリストに追加
#         for idx, row in importance_df_temp.iterrows():
#             feature = row['Feature']
#             normalized_importance = row['Normalized_Importance']
#             importance_list.append({'Feature': feature, 'Normalized_Importance': normalized_importance})

#     # DataFrameを作成
#     importance_df = pd.DataFrame(importance_list)

#     # 各特徴量の正規化された重要度を合計
#     final_importance_df = importance_df.groupby('Feature', as_index=False).sum()

#     # 重要度でソート
#     final_importance_df = final_importance_df.sort_values(by='Normalized_Importance', ascending=False)

#     # 最終的な正規化された重要度を表示
#     print(model_name)
#     print(final_importance_df)

In [64]:
# import pandas as pd

# models = [
#     ctb1_models,
#     lgb1_models,
#     ctb2_models,
#     lgb2_models,
#     ctb3_models,
#     lgb3_models,
#     cox1_models,
#     cox2_models
# ]

# models_name = [
#     "ctb1_models",
#     "lgb1_models",
#     "ctb2_models",
#     "lgb2_models",
#     "ctb3_models",
#     "lgb3_models",
#     "cox1_models",
#     "cox2_models"
# ]

# # 不要な列をドロップしたあとの "学習に使う特徴量" を取得
# features = train_data.drop(
#     ['ID', 'efs', 'efs_time', 'target1', 'target2', 'target3', 'target4'],
#     axis=1
# ).columns

# for module_models, model_name in zip(models, models_name):
#     # foldごとの特徴量重要度データフレームをまとめるリスト
#     fold_importance_dfs = []

#     # クロスバリデーションの各モデルを順番に取り出す
#     for model in module_models:
#         # この fold の生の重要度を取得
#         importance = model.feature_importances_

#         # DataFrameに変換
#         importance_df_temp = pd.DataFrame({
#             'Feature': features,
#             'Importance': importance
#         })

#         # この fold 内での合計重要度
#         total_importance = importance_df_temp['Importance'].sum()

#         # fold 内で正規化（sum=1 となるようにする）
#         # 万が一、total_importance が 0 の場合はエラー回避
#         if total_importance == 0:
#             importance_df_temp['Normalized_Importance'] = 0
#         else:
#             importance_df_temp['Normalized_Importance'] = (
#                 importance_df_temp['Importance'] / total_importance
#             )

#         # この fold の結果をリストに追加
#         fold_importance_dfs.append(importance_df_temp[['Feature', 'Normalized_Importance']])

#     # 全 fold の重要度データフレームを結合
#     all_folds_importance = pd.concat(fold_importance_dfs, axis=0)

#     # 特徴量ごとに「平均の正規化重要度」を計算
#     # sum ではなく mean をとる方が、fold 数によらず比較しやすい
#     final_importance_df = (
#         all_folds_importance
#         .groupby('Feature', as_index=False)['Normalized_Importance']
#         .mean()
#         .sort_values(by='Normalized_Importance', ascending=False)
#     )

#     # 結果を表示
#     print(f"Feature Importance for {model_name}")
#     print(final_importance_df)
#     print("-" * 50)


In [65]:
 # データフレームに変換
importance_df = pd.DataFrame(columns = ['Feature', 'Importance'])
importance_df

,Feature,Importance


In [66]:
cox1_preds = md.infer_model(test_data, cox1_models, 'target4')

In [67]:
cox2_preds = md.infer_model(test_data, cox2_models, 'target4')

<p style="background-color: rgb(247, 230, 202); font-size: 300%; text-align: center; border-radius: 40px 40px; color: rgb(162, 87, 79); font-weight: bold; font-family: 'Roboto'; border: 4px solid rgb(162, 87, 79);">Ensemble Model</p>

<div style="background-color: rgb(247, 230, 202); border: 4px solid rgb(162, 87, 79); border-radius: 40px; padding: 20px; font-family: 'Roboto'; color: rgb(162, 87, 79); text-align: left; font-size: 140%;">
    <b>Calculate C-Index score for Ensemble model using Out-of-Fold (OOF) predictions.</b>
</div>

In [68]:
oof_preds = [
    ctb1_oof_preds,
    lgb1_oof_preds,
    xgb1_oof_preds,
    ctb2_oof_preds,
    lgb2_oof_preds,
    xgb2_oof_preds,
    ctb3_oof_preds,
    lgb3_oof_preds,
    xgb3_oof_preds,
    cox1_oof_preds,
    cox2_oof_preds
]

In [69]:
ranked_oof_preds = np.array([rankdata(p) for p in oof_preds])

In [70]:
# ensemble_oof_preds = np.dot(CFG.weights, ranked_oof_preds)

In [71]:
# md.targets.validate_model(ensemble_oof_preds, 'Ensemble Model')

In [72]:
# best_score = 0
# best_i = 0
# best_k = 0
# for i in range(51):
#     for k in range(51-i):
#         for j in range(51-i-k):
#             if k >= 0 and i >= 0 and j >= 0 and 51-i-k-j >= 0:
#                 CFG.weights = [i, i, k, k, j, j, 101-i-k-j, 101-i-k-j]
#                 ensemble_oof_preds = np.dot(CFG.weights, ranked_oof_preds)
#                 curent_score = md.targets.validate_model(ensemble_oof_preds, 'Ensemble Model')
#                 if best_score < curent_score:
#                     best_i = i
#                     best_k = k
#                     best_j = j
#                     best_score = curent_score

# print(f"best_score->{best_score}, best_i->{best_i}, best_k->{best_k}, best_j->{best_j}")
# CFG.weights = [best_i, best_i, best_k, best_k, best_j, best_j, 101-best_i-best_k-best_j, 101-best_i-best_k-best_j]

In [73]:
import optuna
# 目的関数の定義
def objective(trial):
    # 各モデルの重みを提案 (0.0～1.0 の範囲)
    weights = [trial.suggest_float(f'weight_{i}', 0.05, 1.0) for i in range(len(ranked_oof_preds))]

    # 重みの正規化（合計が1になるようにスケーリング）
    weights = np.array(weights)
    weights /= np.sum(weights)

    # アンサンブル予測を計算 (加重平均)
    ensemble_oof_preds = np.dot(weights, ranked_oof_preds)

    curent_score = md.targets.validate_model(ensemble_oof_preds, 'Ensemble Model')

    return curent_score  # 最大化が目標

# Optunaによる最適化
study = optuna.create_study(direction='maximize')  # AUCを最大化
study.optimize(objective, n_trials=200)

# 最適な重み
print("Best Weights:", study.best_params)
print("Best AUC:", study.best_value)

CFG.weights = [1 for _ in range(len(ranked_oof_preds))]
np.array(CFG.weights)

for i in range(len(CFG.weights)):
    CFG.weights[i] = study.best_params[f'weight_{i}']

[I 2025-02-21 09:26:58,643] A new study created in memory with name: no-name-d639b32c-6e48-4505-ae44-d948de0ab69d
[I 2025-02-21 09:26:59,040] Trial 0 finished with value: 0.6835513144085185 and parameters: {'weight_0': 0.6500797215314316, 'weight_1': 0.7314243335852748, 'weight_2': 0.45417962623000646, 'weight_3': 0.9334112195456278, 'weight_4': 0.06519243260375537, 'weight_5': 0.86382116059298, 'weight_6': 0.6753611600394537, 'weight_7': 0.8180828690717318, 'weight_8': 0.8858666699335332, 'weight_9': 0.5557806293549048, 'weight_10': 0.9434588420716554}. Best is trial 0 with value: 0.6835513144085185.
[I 2025-02-21 09:26:59,432] Trial 1 finished with value: 0.6821292685360433 and parameters: {'weight_0': 0.990906291282041, 'weight_1': 0.5681900715435655, 'weight_2': 0.7548982834395311, 'weight_3': 0.07084220239531926, 'weight_4': 0.06827005454515235, 'weight_5': 0.9557814509064306, 'weight_6': 0.2619468349745099, 'weight_7': 0.7615257661450807, 'weight_8': 0.67245084814134, 'weight_9':

Best Weights: {'weight_0': 0.9579744499801015, 'weight_1': 0.1332286455044081, 'weight_2': 0.07504699681137972, 'weight_3': 0.1285688449482033, 'weight_4': 0.8467056635451448, 'weight_5': 0.6210377798619748, 'weight_6': 0.07166809686218012, 'weight_7': 0.23778688844679563, 'weight_8': 0.8606858657243919, 'weight_9': 0.07276515921476073, 'weight_10': 0.6895436702753052}
Best AUC: 0.6850149383578561


In [74]:
# oof_preds_name = [
#     "ctb1_oof_preds",
#     "lgb1_oof_preds",
#     "ctb2_oof_preds",
#     "lgb2_oof_preds",
#     "ctb3_oof_preds",
#     "lgb3_oof_preds",
#     "cox1_oof_preds",
#     "cox2_oof_preds"
# ]

# for i, oof_pred in enumerate(oof_preds):
#     means = []
#     for pred in oof_pred:
#         means.append(np.mean(pred))

#     print(f"{oof_preds_name[i]}->{np.mean(means)}")


<div style="background-color: rgb(247, 230, 202); border: 4px solid rgb(162, 87, 79); border-radius: 40px; padding: 20px; font-family: 'Roboto'; color: rgb(162, 87, 79); text-align: left; font-size: 140%;">
    <b>Ensemble predictions for the test data.</b>
</div>

In [75]:
preds = [
    ctb1_preds,
    lgb1_preds,
    xgb1_preds,
    ctb2_preds,
    lgb2_preds,
    xgb2_preds,
    ctb3_preds,
    lgb3_preds,
    xgb3_preds,
    cox1_preds,
    cox2_preds
]

In [76]:
# rankdataデータセットの要素を順位付けするための関数です。この関数を使用することで、数値データをランキング形式に変換
# 例えば、リスト [10, 20, 20, 30] に対してランク付けを行うと [1.0, 3.0, 3.0, 4.0] という結果が得られます。

ranked_preds = np.array([rankdata(p) for p in preds])

In [77]:
ensemble_preds = np.dot(CFG.weights, ranked_preds)

In [78]:
subm_data = pd.read_csv(CFG.subm_path)
subm_data['prediction'] = ensemble_preds

In [79]:
subm_data.to_csv('submission.csv', index=False)
display(subm_data.head())

,ID,prediction
0,28800,7.833253
1,28801,14.085036
2,28802,6.251783
